# Universal vs Process-specific vs NonCHM CpGrich

## Obtain three group annotations

In [3]:
%%bash
anPATH=/mnt/Storage/home/yanghui/annotations
dataPATH_ymw=/mnt/Storage/home/wangyiman/CHMsInOtherContexts/CellStateTransition/CHMOrganization
mkdir -p Universal_ProcessSpecific_NonCHMsCpGrich;cd Universal_ProcessSpecific_NonCHMsCpGrich
ln -s ${dataPATH_ymw}/Universal_specific/Universal.CHM.bed UniversalCHMs.bed # 2,677
cat ${dataPATH_ymw}/Universal_specific/*Specific.CHM.bed | sort -k1,1 -k2,2n | mergeBed -i - -d 2000 > ProcessSpecificCHMs.bed # 6,980
awk 'BEGIN{FS=OFS="\t"}{if($4>=30){print $1, $2, $3;}}' ${anPATH}/mm10/Bins/mm10.b1kb.euchr.CpGnumber > mm10.b1kb.euchr.CpGrich.bed # 41,427
cat ${dataPATH_ymw}/*/merged.CHM.bed | windowBed -w 2000 -v -a mm10.b1kb.euchr.CpGrich.bed -b - > NonCHMsCpGrich.bed # 22,298

## Region enrichment

### Don't divide according to chromatin states

#### Obtain region enrichment details

In [5]:
%%bash
shPATH=/mnt/Storage/home/yanghui/scripts/shell
anPATH=/mnt/Storage/home/yanghui/annotations
dirPATH=/mnt/Storage/home/yanghui/imprinting/result.2021/Exploring/CHMsInOtherContexts/CellStateTransition/CHMOrganization/Universal_specific/Features/Repeats
cd ${dirPATH}/Universal_ProcessSpecific_NonCHMsCpGrich
for group in UniversalCHMs ProcessSpecificCHMs NonCHMsCpGrich
do
    bash ${shPATH}/regionEnrichment_userDefined.sh \
        "${dirPATH}/Universal_ProcessSpecific_NonCHMsCpGrich" \
        "mm10_euch" \
        "${anPATH}/mm10/mm10_euch.chrom.sizes" \
        "${group}.bed" \
        "${anPATH}/mm10/RegionEnrichment/mm10_euch.CGI_Unmasked.merged.bed;${anPATH}/mm10/RegionEnrichment/mm10_euch.promoter.merged.bed;${anPATH}/mm10/RegionEnrichment/mm10_euch.geneBody.merged.bed;${anPATH}/mm10/RegionEnrichment/mm10_euch.exon.merged.bed;${anPATH}/mm10/RegionEnrichment/mm10_euch.intron.merged.bed;${anPATH}/mm10/RegionEnrichment/mm10_euch.LTR.merged.bed;${anPATH}/mm10/RegionEnrichment/mm10_euch.LINE.merged.bed;${anPATH}/mm10/RegionEnrichment/mm10_euch.SINE.merged.bed" \
        "regionEnrich_${group}"
done # for group end

null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 


#### Plot together

In [7]:
%%bash
dirPATH=/mnt/Storage/home/yanghui/imprinting/result.2021/Exploring/CHMsInOtherContexts/CellStateTransition/CHMOrganization/Universal_specific/Features/Repeats
rPATH=/mnt/Storage/home/yanghui/scripts/R
cd ${dirPATH}/Universal_ProcessSpecific_NonCHMsCpGrich

# # Plot together
# echo -e "xValue\tyValue\tgroup" > regionEnrich.ggplot_basic.txt
# for group in UniversalCHMs ProcessSpecificCHMs NonCHMsCpGrich
# do
#     tail -n +2 regionEnrich_${group}.EnrichScore.txt | \
#         awk -v GROUP=${group} 'BEGIN{FS=OFS="\t"}{\
#         split($1, a, "_");\
#         print a[1], $2, GROUP;}' >> regionEnrich.ggplot_basic.txt
# done # for group end

Rscript ${rPATH}/ggplot_barplot.r \
    "regionEnrich.ggplot_basic.txt" \
    "regionEnrich.ggplot_barplot.pdf" \
    "Region enrichment" \
    "" \
    "Enrichment score" \
    "-6;6.2;2" \
    "CGI;promoter;geneBody;exon;intron;LTR;LINE;SINE" \
    "UniversalCHMs;ProcessSpecificCHMs;NonCHMsCpGrich" \
    "dodge" \
    "7" \
    "3" \
    "1" \
    "F"

null device 
          1 


### Divide according to chromatin states

#### Obtain region enrichment details

In [35]:
%%bash
shPATH=/mnt/Storage/home/yanghui/scripts/shell
anPATH=/mnt/Storage/home/yanghui/annotations
dirPATH=/mnt/Storage/home/yanghui/imprinting/result.2021/Exploring/CHMsInOtherContexts/CellStateTransition/CHMOrganization/Universal_specific/Features/Repeats
dataPATH_compartment=/mnt/Storage/home/wangyiman/CHMsInOtherContexts/CellStateTransition/CHMOrganization/Universal_specific/Features/ChromatinStates/Cavalli_Cell2017_ABcompartment
cd ${dirPATH}/Universal_ProcessSpecific_NonCHMsCpGrich
for group in UniversalCHMs ProcessSpecificCHMs NonCHMsCpGrich
do
    intersectBed -u -a ${group}.bed -b ${dataPATH_compartment}/AB_compartment_intersectA_Cavalli_Cell2017.bed > ${group}.cmpt_A.bed # 
    intersectBed -u -a ${group}.bed -b ${dataPATH_compartment}/AB_compartment_intersectB_Cavalli_Cell2017.bed | \
        intersectBed -v -a - -b ${group}.cmpt_A.bed > ${group}.cmpt_B.bed # 
    
    for cmpt in A B
    do
        bash ${shPATH}/regionEnrichment_userDefined.sh \
            "${dirPATH}/Universal_ProcessSpecific_NonCHMsCpGrich" \
            "mm10_euch" \
            "${anPATH}/mm10/mm10_euch.chrom.sizes" \
            "${group}.cmpt_${cmpt}.bed" \
            "${anPATH}/mm10/RegionEnrichment/mm10_euch.CGI_Unmasked.merged.bed;${anPATH}/mm10/RegionEnrichment/mm10_euch.promoter.merged.bed;${anPATH}/mm10/RegionEnrichment/mm10_euch.geneBody.merged.bed;${anPATH}/mm10/RegionEnrichment/mm10_euch.exon.merged.bed;${anPATH}/mm10/RegionEnrichment/mm10_euch.intron.merged.bed;${anPATH}/mm10/RegionEnrichment/mm10_euch.LTR.merged.bed;${anPATH}/mm10/RegionEnrichment/mm10_euch.LINE.merged.bed;${anPATH}/mm10/RegionEnrichment/mm10_euch.SINE.merged.bed" \
            "regionEnrich_${group}_${cmpt}"
    done # for cmpt end
done # for group end

null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 


#### Plot together

In [36]:
%%bash
dirPATH=/mnt/Storage/home/yanghui/imprinting/result.2021/Exploring/CHMsInOtherContexts/CellStateTransition/CHMOrganization/Universal_specific/Features/Repeats
rPATH=/mnt/Storage/home/yanghui/scripts/R
cd ${dirPATH}/Universal_ProcessSpecific_NonCHMsCpGrich

# Plot together
echo -e "xValue\tyValue\tgroup" > regionEnrich_cmpt.ggplot_basic.txt
for group in UniversalCHMs ProcessSpecificCHMs NonCHMsCpGrich
do
	for cmpt in A B
	do
    	tail -n +2 regionEnrich_${group}_${cmpt}.EnrichScore.txt | \
    	    awk -v GROUP="${group} ov ${cmpt}" 'BEGIN{FS=OFS="\t"}{\
    	    split($1, a, "_");\
    	    print a[1], $2, GROUP;}' >> regionEnrich_cmpt.ggplot_basic.txt
    done # for cmpt end
done # for group end

Rscript ${rPATH}/ggplot_barplot.r \
    "regionEnrich_cmpt.ggplot_basic.txt" \
    "regionEnrich_cmpt.ggplot_barplot.pdf" \
    "Region enrichment" \
    "" \
    "Enrichment score" \
    "-6;6.2;2" \
    "CGI;promoter;geneBody;exon;intron;LTR;LINE;SINE" \
    "UniversalCHMs ov A;UniversalCHMs ov B;ProcessSpecificCHMs ov A;ProcessSpecificCHMs ov B;NonCHMsCpGrich ov A;NonCHMsCpGrich ov B" \
    "dodge" \
    "7" \
    "3" \
    "1" \
    "F"

null device 
          1 


## Repeats subfamily enrichment

### Don't divide according to chromatin states

#### Obtain enrichment score

In [12]:
%%bash
dirPATH=/mnt/Storage/home/yanghui/imprinting/result.2021/Exploring/CHMsInOtherContexts/CellStateTransition/CHMOrganization/Universal_specific/Features/Repeats
shPATH=/mnt/Storage/home/yanghui/scripts/shell
cd ${dirPATH}/Universal_ProcessSpecific_NonCHMsCpGrich
# mkdir -p LTR_Subfamily LINE_Subfamily SINE_Subfamily

for group in UniversalCHMs ProcessSpecificCHMs NonCHMsCpGrich
do
    for repeatType in LTR LINE SINE
    do
        bash ${shPATH}/regionEnrichment_${repeatType}_mm10_euch.sh \
            "${dirPATH}/Universal_ProcessSpecific_NonCHMsCpGrich/${repeatType}_Subfamily" \
            "${dirPATH}/Universal_ProcessSpecific_NonCHMsCpGrich/${group}.bed" \
            "${group}"
    done # for repeatType end
done # for group end

null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 

Runtime error (func=(main), adr=3): Divide by zero
Runtime error (func=(main), adr=3): Divide by zero
Runtime error (func=(main), adr=3): Divide by zero


#### Prepare for sina plot

In [45]:
%%bash
dirPATH=/mnt/Storage/home/yanghui/imprinting/result.2021/Exploring/CHMsInOtherContexts/CellStateTransition/CHMOrganization/Universal_specific/Features/Repeats

for repeatType in LTR LINE SINE
do
    cd ${dirPATH}/Universal_ProcessSpecific_NonCHMsCpGrich/${repeatType}_Subfamily
    echo -e "xValue\tyValue\tgroup\tcol\tlabel" > enrichmentScore.ggplot_sina.txt
    for group in UniversalCHMs ProcessSpecificCHMs NonCHMsCpGrich
    do
        for fmlEnrich in $(ls ${group}.EnrichScore_${repeatType}_*.txt)
        do
            fmlName=$(echo ${fmlEnrich#*EnrichScore_${repeatType}_*})
            fmlName=$(echo ${fmlName%.txt})
            tail -n +2 ${fmlEnrich} | grep -v "inf" | \
                awk -v XVALUE=${group} -v GROUP=${repeatType} -v COL=${fmlName} 'BEGIN{FS=OFS="\t"}{\
                print XVALUE, $2, GROUP, COL, $1;}' >> enrichmentScore.ggplot_sina.txt
        done # for fmlEnrich end
    done # for group end
done # for repeatType end

#### Sina plot

In [47]:
%%bash
dirPATH=/mnt/Storage/home/yanghui/imprinting/result.2021/Exploring/CHMsInOtherContexts/CellStateTransition/CHMOrganization/Universal_specific/Features/Repeats
rPATH=/mnt/Storage/home/yanghui/scripts/R

LTR_sina(){
    cd ${dirPATH}/Universal_ProcessSpecific_NonCHMsCpGrich/LTR_Subfamily
    Rscript ${rPATH}/ggplot_sina_colorfull.r \
        enrichmentScore.ggplot_sina.txt \
        enrichmentScore_filtered.ggplot_sina.pdf \
        "Enrichment score" \
        "-12;8;4" \
        "UniversalCHMs;ProcessSpecificCHMs;NonCHMsCpGrich" \
        "LTR" \
        "ERV1;ERVK;ERVL;ERVL-MaLR;Gypsy" \
        "#CE0013;#C7A609;#64C0AB;#A14C94;#1187CD" \
        "0.2" \
        "0.75" \
        "1" \
        "5" \
        "5"
}
LTR_sina

LINE_sina(){
    cd ${dirPATH}/Universal_ProcessSpecific_NonCHMsCpGrich/LINE_Subfamily
    Rscript ${rPATH}/ggplot_sina_colorfull.r \
        enrichmentScore.ggplot_sina.txt \
        enrichmentScore_filtered.ggplot_sina.pdf \
        "Enrichment score" \
        "-12;8;4" \
        "UniversalCHMs;ProcessSpecificCHMs;NonCHMsCpGrich" \
        "LINE" \
        "CR1;Dong-R4;Jockey;L1;L1-Tx1;L2;Penelope;RTE-BovB;RTE-X" \
        "#CE0013;#16557A;#C7A609;#87C232;#64C0AB;#A14C94;#8B7E75;#1187CD;#000000" \
        "0.2" \
        "0.75" \
        "1" \
        "5" \
        "5"
}
LINE_sina

SINE_sina(){
    cd ${dirPATH}/Universal_ProcessSpecific_NonCHMsCpGrich/SINE_Subfamily
    Rscript ${rPATH}/ggplot_sina_colorfull.r \
        enrichmentScore.ggplot_sina.txt \
        enrichmentScore_filtered.ggplot_sina.pdf \
        "Enrichment score" \
        "-12;8;4" \
        "UniversalCHMs;ProcessSpecificCHMs;NonCHMsCpGrich" \
        "SINE" \
        "5S-Deu-L2;Alu;B2;B4;ID;MIR;tRNA;tRNA-Deu;tRNA-RTE" \
        "#CE0013;#16557A;#C7A609;#87C232;#64C0AB;#A14C94;#8B7E75;#1187CD;#000000" \
        "0.2" \
        "0.75" \
        "1" \
        "5" \
        "5"
}
SINE_sina

# R: Cant 

[1] "#CE0013" "#C7A609" "#64C0AB" "#A14C94" "#1187CD"
[1] "ERV1"      "ERVK"      "ERVL"      "ERVL-MaLR" "Gypsy"    

  1   2   3   4   5 
157 484 116 176  11 
null device 
          1 
[1] "#CE0013" "#16557A" "#C7A609" "#87C232" "#64C0AB" "#A14C94" "#8B7E75"
[8] "#1187CD" "#000000"
[1] "CR1"      "Dong-R4"  "Jockey"   "L1"       "L1-Tx1"   "L2"       "Penelope"
[8] "RTE-BovB" "RTE-X"   

  1   3   4   6   7   8   9 
 17   1 298  23   3   3   4 
null device 
          1 
[1] "#CE0013" "#16557A" "#C7A609" "#87C232" "#64C0AB" "#A14C94" "#8B7E75"
[8] "#1187CD" "#000000"
[1] "5S-Deu-L2" "Alu"       "B2"        "B4"        "ID"        "MIR"      
[7] "tRNA"      "tRNA-Deu"  "tRNA-RTE" 

 1  2  3  4  5  6  7  8  9 
 1 46 15 12 10 15  2  1  3 
null device 
          1 


#### Scatter plot

In [68]:
%%bash
dirPATH=/mnt/Storage/home/yanghui/imprinting/result.2021/Exploring/CHMsInOtherContexts/CellStateTransition/CHMOrganization/Universal_specific/Features/Repeats
rPATH=/mnt/Storage/home/yanghui/scripts/R

LTR_F(){
    cd ${dirPATH}/Universal_ProcessSpecific_NonCHMsCpGrich/LTR_Subfamily
    echo -e "xValue\tyValue\tgroup\tshape\tsubfamily" > enrichDetails.ggplot_scatter.txt # group: ERV1...; shape: UniversalCHMs
    for group in UniversalCHMs ProcessSpecificCHMs NonCHMsCpGrich
    do
        for fml in ERV1 ERVK ERVL ERVL-MaLR Gypsy
        do
            paste ${group}.overlaped_LTR_${fml}.txt ${group}.EnrichScore_LTR_${fml}.txt | \
                tail -n +2 | awk -v GROUP=${fml} -v SHAPE=${group} 'BEGIN{FS=OFS="\t"}{if($1==$4 && $5!="-inf"){print $3*100, $5, GROUP, SHAPE, $1;}}' >> enrichDetails.ggplot_scatter.txt
        done # for fml end
    done # for group end
    
    
    
    Rscript ${rPATH}/ggplot_scatterplot_basic.r \
        enrichDetails.ggplot_scatter.txt \
        enrichDetails \
        "Overlapped ratio (%)" \
        "Enrichment score" \
        "0;80;20" \
        "-12;8;4" \
        "ERV1;ERVK;ERVL;ERVL-MaLR;Gypsy" \
        "T;1;0" \
        "0.75" \
        "5" \
        "5" \
        "Set1"
}
LTR_F

LINE_F(){
    cd ${dirPATH}/Universal_ProcessSpecific_NonCHMsCpGrich/LINE_Subfamily
    echo -e "xValue\tyValue\tgroup\tshape\tsubfamily" > enrichDetails.ggplot_scatter.txt # group: ERV1...; shape: UniversalCHMs
    for group in UniversalCHMs ProcessSpecificCHMs NonCHMsCpGrich
    do
        for fml in CR1 Dong-R4 Jockey L1 L1-Tx1 L2 Penelope RTE-BovB RTE-X
        do
            paste ${group}.overlaped_LINE_${fml}.txt ${group}.EnrichScore_LINE_${fml}.txt | \
                tail -n +2 | awk -v GROUP=${fml} -v SHAPE=${group} 'BEGIN{FS=OFS="\t"}{if($1==$4 && $5!="-inf"){print $3*100, $5, GROUP, SHAPE, $1;}}' >> enrichDetails.ggplot_scatter.txt
        done # for fml end
    done # for group end
    
    
    
    Rscript ${rPATH}/ggplot_scatterplot_basic.r \
        enrichDetails.ggplot_scatter.txt \
        enrichDetails \
        "Overlapped ratio (%)" \
        "Enrichment score" \
        "0;100;20" \
        "-12;8;4" \
        "CR1;Dong-R4;Jockey;L1;L1-Tx1;L2;Penelope;RTE-BovB;RTE-X" \
        "T;1;0" \
        "0.75" \
        "5" \
        "5" \
        "Set1"
}
LINE_F

SINE_F(){
    cd ${dirPATH}/Universal_ProcessSpecific_NonCHMsCpGrich/SINE_Subfamily
    echo -e "xValue\tyValue\tgroup\tshape\tsubfamily" > enrichDetails.ggplot_scatter.txt # group: ERV1...; shape: UniversalCHMs
    for group in UniversalCHMs ProcessSpecificCHMs NonCHMsCpGrich
    do
        for fml in 5S-Deu-L2 Alu B2 B4 ID MIR tRNA tRNA-Deu tRNA-RTE
        do
            paste ${group}.overlaped_SINE_${fml}.txt ${group}.EnrichScore_SINE_${fml}.txt | \
                tail -n +2 | awk -v GROUP=${fml} -v SHAPE=${group} 'BEGIN{FS=OFS="\t"}{if($1==$4 && $5!="-inf"){print $3*100, $5, GROUP, SHAPE, $1;}}' >> enrichDetails.ggplot_scatter.txt
        done # for fml end
    done # for group end
    
    
    
    Rscript ${rPATH}/ggplot_scatterplot_basic.r \
        enrichDetails.ggplot_scatter.txt \
        enrichDetails \
        "Overlapped ratio (%)" \
        "Enrichment score" \
        "0;100;20" \
        "-12;8;4" \
        "5S-Deu-L2;Alu;B2;B4;ID;MIR;tRNA;tRNA-Deu;tRNA-RTE" \
        "T;1;0" \
        "0.75" \
        "5" \
        "5" \
        "Set1"
}
SINE_F

null device 
          1 
null device 
          1 
null device 
          1 


#### Pie plot showing enriched subfamily

##### Prepare for pie plot

In [19]:
%%bash
dirPATH=/mnt/Storage/home/yanghui/imprinting/result.2021/Exploring/CHMsInOtherContexts/CellStateTransition/CHMOrganization/Universal_specific/Features/Repeats

for repeatType in LTR LINE SINE
do
    cd ${dirPATH}/Universal_ProcessSpecific_NonCHMsCpGrich/${repeatType}_Subfamily
    for group in UniversalCHMs ProcessSpecificCHMs NonCHMsCpGrich
    do
        echo -e "value\tgroup" > ${group}.enriched.ggplot_pieplot.txt
        for fmlEnrich in $(ls ${group}.EnrichScore_${repeatType}_*.txt)
        do
            fmlName=$(echo ${fmlEnrich#*EnrichScore_${repeatType}_*})
            fmlName=$(echo ${fmlName%.txt})
            tail -n +2 ${fmlEnrich} | \
                awk -v GROUP=${fmlName} 'BEGIN{FS=OFS="\t";TOTAL=0;}{\
                if($2>=1){TOTAL+=1;}}END{print TOTAL, GROUP;}' >> ${group}.enriched.ggplot_pieplot.txt
        done # for fmlEnrich end
    done # for group end
done # for repeatType end

##### Pie plot

In [20]:
%%bash
dirPATH=/mnt/Storage/home/yanghui/imprinting/result.2021/Exploring/CHMsInOtherContexts/CellStateTransition/CHMOrganization/Universal_specific/Features/Repeats
rPATH=/mnt/Storage/home/yanghui/scripts/R

LTR_pie(){
    cd ${dirPATH}/Universal_ProcessSpecific_NonCHMsCpGrich/LTR_Subfamily
    for group in UniversalCHMs ProcessSpecificCHMs NonCHMsCpGrich
    do
        Rscript ${rPATH}/ggplot_pie.r \
        ${group}.enriched.ggplot_pieplot.txt \
        ${group} \
        "ERV1;ERVK;ERVL;ERVL-MaLR;Gypsy" \
        "#CE0013;#C7A609;#64C0AB;#A14C94;#1187CD" \
        T \
        3.5 \
        3.5
    done # for group end
}
LTR_pie

LINE_pie(){
    cd ${dirPATH}/Universal_ProcessSpecific_NonCHMsCpGrich/LINE_Subfamily
    for group in UniversalCHMs ProcessSpecificCHMs NonCHMsCpGrich
    do
        Rscript ${rPATH}/ggplot_pie.r \
        ${group}.enriched.ggplot_pieplot.txt \
        ${group} \
        "CR1;Dong-R4;Jockey;L1;L1-Tx1;L2;Penelope;RTE-BovB;RTE-X" \
        "#CE0013;#16557A;#C7A609;#87C232;#64C0AB;#A14C94;#8B7E75;#1187CD;#000000" \
        T \
        3.5 \
        3.5
    done # for group end
}
LINE_pie

SINE_pie(){
    cd ${dirPATH}/Universal_ProcessSpecific_NonCHMsCpGrich/SINE_Subfamily
    for group in UniversalCHMs ProcessSpecificCHMs NonCHMsCpGrich
    do
        Rscript ${rPATH}/ggplot_pie.r \
        ${group}.enriched.ggplot_pieplot.txt \
        ${group} \
        "5S-Deu-L2;Alu;B2;B4;ID;MIR;tRNA;tRNA-Deu;tRNA-RTE" \
        "#CE0013;#16557A;#C7A609;#87C232;#64C0AB;#A14C94;#8B7E75;#1187CD;#000000" \
        T \
        3.5 \
        3.5
    done # for group end
}
SINE_pie

null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 


Warning messages:
1: Removed 9 rows containing missing values (position_stack). 
2: Removed 9 rows containing missing values (geom_text). 
Warning messages:
1: Removed 9 rows containing missing values (position_stack). 
2: Removed 9 rows containing missing values (geom_text). 


#### Overlap between enriched subfamily

In [21]:
%%bash
dirPATH=/mnt/Storage/home/yanghui/imprinting/result.2021/Exploring/CHMsInOtherContexts/CellStateTransition/CHMOrganization/Universal_specific/Features/Repeats

for repeatType in LTR LINE SINE
do
    cd ${dirPATH}/Universal_ProcessSpecific_NonCHMsCpGrich/${repeatType}_Subfamily
    for group in UniversalCHMs ProcessSpecificCHMs NonCHMsCpGrich
    do
        cat ${group}.EnrichScore_${repeatType}_*.txt | grep -v "EnrichScore" | \
            awk 'BEGIN{FS=OFS="\t";}{if($2!="-inf" && $2>=1){print $1;}}' | sort -u > ${group}.subfml_enriched.txt # 
        cat ${group}.EnrichScore_${repeatType}_*.txt | grep -v "EnrichScore" | \
            awk 'BEGIN{FS=OFS="\t";}{if($2=="-inf" || $2<1){print $1;}}' | sort -u | grep -wvf ${group}.subfml_enriched.txt - > ${group}.subfml_nonenriched.txt # 
    done # for group end
done # for repeatType end

#### Conservation score

##### Multiple groups according to overlap

###### Group according to overlap

In [22]:
%%bash
dirPATH=/mnt/Storage/home/yanghui/imprinting/result.2021/Exploring/CHMsInOtherContexts/CellStateTransition/CHMOrganization/Universal_specific/Features/Repeats
for repeatType in LTR LINE SINE
do
    cd ${dirPATH}/Universal_ProcessSpecific_NonCHMsCpGrich/${repeatType}_Subfamily
    grep -wf UniversalCHMs.subfml_enriched.txt ProcessSpecificCHMs.subfml_enriched.txt > BothCHMs.subfml_enriched.txt #
    grep -vwf UniversalCHMs.subfml_enriched.txt ProcessSpecificCHMs.subfml_enriched.txt > ProcessSpecificCHMsOnly.subfml_enriched.txt # 
    grep -vwf ProcessSpecificCHMs.subfml_enriched.txt UniversalCHMs.subfml_enriched.txt > UniversalCHMsOnly.subfml_enriched.txt # 
    cat UniversalCHMs.EnrichScore_${repeatType}_*.txt | grep -v "EnrichScore" | \
        cut -f 1 | sort -u > all.subfml.txt
    cat *.subfml_enriched.txt | sort -u | grep -vwf - all.subfml.txt > subfml_nonenriched.txt # 
done # for repeatType end

###### Obtain corresponding annotation

In [24]:
%%bash
anPATH=/mnt/Storage/home/yanghui/annotations
dirPATH=/mnt/Storage/home/yanghui/imprinting/result.2021/Exploring/CHMsInOtherContexts/CellStateTransition/CHMOrganization/Universal_specific/Features/Repeats

for repeatType in LTR LINE SINE
do
    cd ${dirPATH}/Universal_ProcessSpecific_NonCHMsCpGrich/${repeatType}_Subfamily
    for group in BothCHMs.subfml_enriched ProcessSpecificCHMsOnly.subfml_enriched UniversalCHMsOnly.subfml_enriched NonCHMsCpGrich.subfml_enriched subfml_nonenriched
    do
        if [ -s ${group}.txt ]
        then
            for subfml in $(cut -f 1 ${group}.txt)
            do
                cat ${anPATH}/mm10/Repeats/${repeatType}/*/mm10.${subfml}.bed | \
                    intersectBed -a - -b ${anPATH}/mm10/mm10_euch.chrom.limits | \
                    cut -f 1-3 >> ${group}.bed
            done # for subfml end
        fi
    done # for group end
done # for repeatType end

###### Obtain average conservation score

In [28]:
%%bash
dirPATH=/mnt/Storage/home/yanghui/imprinting/result.2021/Exploring/CHMsInOtherContexts/CellStateTransition/CHMOrganization/Universal_specific/Features/Repeats
anPATH=/mnt/Storage/home/yanghui/annotations
shPATH=/mnt/Storage/home/yanghui/scripts/shell
rPATH=/mnt/Storage/home/yanghui/scripts/R

for repeatType in LTR LINE SINE
do
    cd ${dirPATH}/Universal_ProcessSpecific_NonCHMsCpGrich/${repeatType}_Subfamily
    # echo -e "xValue\tyValue\tgroup" > phastCons60wayPlacental.ggplot_basic.txt
    # for group in BothCHMs.subfml_enriched ProcessSpecificCHMsOnly.subfml_enriched UniversalCHMsOnly.subfml_enriched NonCHMsCpGrich.subfml_enriched subfml_nonenriched
    # do
    #     if [ -f ${group}.bed ]
    #     then
    #         bash ${shPATH}/averageSignalInRegion.sh \
    #             ${group}.bed \
    #             ${anPATH}/mm10/Conservation/mm10.60way.phastCons60wayPlacental.bw \
    #             1 \
    #             ${group}.phastConsPlacental.txt
    #         
    #         awk -v XVALUE=${group} -v GROUP=${repeatType} 'BEGIN{FS=OFS="\t"}{print XVALUE, $4, GROUP;}' \
    #             ${group}.phastConsPlacental.txt >> phastCons60wayPlacental.ggplot_basic.txt
    #     fi
    # done # for group end
    
    # T: executed in 6m 47s, finished 17:02:44 2023-01-02
    
    # # Violin plot
    # Rscript ${rPATH}/ggplot_violin.r \
    #     phastCons60wayPlacental.ggplot_basic.txt \
    #     phastCons60wayPlacental.ggplot_violin.pdf \
    #     ${repeatType} \
    #     "" \
    #     "Conservation score" \
    #     0 \
    #     1 \
    #     0.2 \
    #     "BothCHMs.subfml_enriched;ProcessSpecificCHMsOnly.subfml_enriched;UniversalCHMsOnly.subfml_enriched;NonCHMsCpGrich.subfml_enriched;subfml_nonenriched" \
    #     "${repeatType}" \
    #     4 \
    #     9
                
    # box plot
    Rscript ${rPATH}/ggplot_boxplot.r \
        phastCons60wayPlacental.ggplot_basic.txt \
        phastCons60wayPlacental.ggplot_boxplot.pdf \
        ${repeatType} \
        "" \
        "Conservation score" \
        0 \
        1 \
        0.2 \
        "BothCHMs.subfml_enriched;ProcessSpecificCHMsOnly.subfml_enriched;UniversalCHMsOnly.subfml_enriched;NonCHMsCpGrich.subfml_enriched;subfml_nonenriched" \
        "${repeatType}" \
        4 \
        9
done # for repeatType end

null device 
          1 
null device 
          1 
[1] "wilcox.test of significance between different xValues"
# A tibble: 10 × 8
   .y.    group1                group2        p   p.adj p.format p.signif method
   <chr>  <chr>                 <chr>     <dbl>   <dbl> <chr>    <chr>    <chr> 
 1 yValue BothCHMs.subfml_enri… Proce… 1.28e-61 1  e-60 < 2e-16  ****     Wilco…
 2 yValue BothCHMs.subfml_enri… Unive… 3.29e-12 2  e-11 3.3e-12  ****     Wilco…
 3 yValue BothCHMs.subfml_enri… NonCH… 1.60e- 6 8  e- 6 1.6e-06  ****     Wilco…
 4 yValue BothCHMs.subfml_enri… subfm… 0        0       < 2e-16  ****     Wilco…
 5 yValue ProcessSpecificCHMsO… Unive… 9.81e- 1 9.8e- 1 0.981    ns       Wilco…
 6 yValue ProcessSpecificCHMsO… NonCH… 2.30e- 2 9.2e- 2 0.023    *        Wilco…
 7 yValue ProcessSpecificCHMsO… subfm… 0        0       < 2e-16  ****     Wilco…
 8 yValue UniversalCHMsOnly.su… NonCH… 3.19e- 2 9.6e- 2 0.032    *        Wilco…
 9 yValue UniversalCHMsOnly.su… subfm… 4.86e-23 3.4e-22 < 2

Warning message:
Removed 1 rows containing missing values (geom_hline). 
Warning message:
Removed 1 rows containing missing values (geom_hline). 
Warning message:
Removed 1 rows containing missing values (geom_hline). 


##### Enriched vs Non-enriched subfamilies

###### Groups

In [79]:
%%bash
dirPATH=/mnt/Storage/home/yanghui/imprinting/result.2021/Exploring/CHMsInOtherContexts/CellStateTransition/CHMOrganization/Universal_specific/Features/Repeats
for repeatType in LTR LINE SINE
do
    cd ${dirPATH}/Universal_ProcessSpecific_NonCHMsCpGrich/${repeatType}_Subfamily
    cat UniversalCHMs.subfml_enriched.txt ProcessSpecificCHMs.subfml_enriched.txt | sort -u > subfml_Enriched.txt #
    grep -vwf subfml_Enriched.txt all.subfml.txt > subfml_NonEnriched.txt #
done # for repeatType end

###### Obtain corresponding annotation

In [30]:
%%bash
anPATH=/mnt/Storage/home/yanghui/annotations
dirPATH=/mnt/Storage/home/yanghui/imprinting/result.2021/Exploring/CHMsInOtherContexts/CellStateTransition/CHMOrganization/Universal_specific/Features/Repeats

for repeatType in LTR LINE SINE
do
    cd ${dirPATH}/Universal_ProcessSpecific_NonCHMsCpGrich/${repeatType}_Subfamily
    for group in Enriched NonEnriched
    do
        if [ -s subfml_${group}.txt ]
        then
            for subfml in $(cut -f 1 subfml_${group}.txt)
            do
                cat ${anPATH}/mm10/Repeats/${repeatType}/*/mm10.${subfml}.bed | \
                    intersectBed -a - -b ${anPATH}/mm10/mm10_euch.chrom.limits | \
                    cut -f 1-3 >> subfml_${group}.bed
            done # for subfml end
        fi
        sort -k1,1 -k2,2n -u subfml_${group}.bed > subfml_${group}.bed.tmp && mv subfml_${group}.bed.tmp subfml_${group}.bed
    done # for group end
done # for repeatType end

sort: cannot read: subfml_Enriched.bed: No such file or directory


###### Obtain average conservation score

In [34]:
%%bash
dirPATH=/mnt/Storage/home/yanghui/imprinting/result.2021/Exploring/CHMsInOtherContexts/CellStateTransition/CHMOrganization/Universal_specific/Features/Repeats
anPATH=/mnt/Storage/home/yanghui/annotations
shPATH=/mnt/Storage/home/yanghui/scripts/shell
rPATH=/mnt/Storage/home/yanghui/scripts/R

for repeatType in LTR LINE
do
    cd ${dirPATH}/Universal_ProcessSpecific_NonCHMsCpGrich/${repeatType}_Subfamily
    # echo -e "xValue\tyValue\tgroup" > phastCons60wayPlacental_subfml.ggplot_basic.txt
    # for group in Enriched NonEnriched
    # do
    #     if [ -f subfml_${group}.bed ]
    #     then
    #         bash ${shPATH}/averageSignalInRegion.sh \
    #             subfml_${group}.bed \
    #             ${anPATH}/mm10/Conservation/mm10.60way.phastCons60wayPlacental.bw \
    #             1 \
    #             subfml_${group}.phastConsPlacental.txt
    #         
    #         awk -v XVALUE=${group} -v GROUP=${repeatType} 'BEGIN{FS=OFS="\t"}{print XVALUE, $4, GROUP;}' \
    #             subfml_${group}.phastConsPlacental.txt >> phastCons60wayPlacental_subfml.ggplot_basic.txt
    #     fi
    # done # for group end
    
    # T: executed in 4m 10s, finished 14:36:19 2023-01-03
    
    # Violin plot
    Rscript ${rPATH}/ggplot_violin.r \
        phastCons60wayPlacental_subfml.ggplot_basic.txt \
        phastCons60wayPlacental_subfml.ggplot_violin.pdf \
        ${repeatType} \
        "" \
        "Conservation score" \
        0 \
        0.6 \
        0.2 \
        "Enriched;NonEnriched" \
        "${repeatType}" \
        3 \
        4
                
    # box plot
    Rscript ${rPATH}/ggplot_boxplot.r \
        phastCons60wayPlacental_subfml.ggplot_basic.txt \
        phastCons60wayPlacental_subfml.ggplot_boxplot.pdf \
        ${repeatType} \
        "" \
        "Conservation score" \
        0 \
        0.6 \
        0.2 \
        "Enriched;NonEnriched" \
        "${repeatType}" \
        3 \
        4
done # for repeatType end

null device 
          1 
null device 
          1 
[1] "wilcox.test of significance between different xValues"
# A tibble: 1 × 8
  .y.    group1   group2          p p.adj p.format p.signif method  
  <chr>  <chr>    <chr>       <dbl> <dbl> <chr>    <chr>    <chr>   
1 yValue Enriched NonEnriched     0     0 <2e-16   ****     Wilcoxon
[1] "t.test of significance between different xValues"
# A tibble: 1 × 8
  .y.    group1   group2          p p.adj p.format p.signif method
  <chr>  <chr>    <chr>       <dbl> <dbl> <chr>    <chr>    <chr> 
1 yValue Enriched NonEnriched     0     0 <2e-16   ****     T-test
null device 
          1 
null device 
          1 
[1] "wilcox.test of significance between different xValues"
# A tibble: 1 × 8
  .y.    group1   group2          p p.adj p.format p.signif method  
  <chr>  <chr>    <chr>       <dbl> <dbl> <chr>    <chr>    <chr>   
1 yValue Enriched NonEnriched     0     0 <2e-16   ****     Wilcoxon
[1] "t.test of significance between different xValue

Warning messages:
1: Removed 5398 rows containing non-finite values (stat_ydensity). 
2: Removed 5398 rows containing non-finite values (stat_boxplot). 
Warning messages:
1: Removed 5398 rows containing non-finite values (stat_boxplot). 
2: Removed 5398 rows containing non-finite values (stat_boxplot). 
3: Removed 1 rows containing missing values (geom_hline). 
Warning messages:
1: Removed 10169 rows containing non-finite values (stat_ydensity). 
2: Removed 10169 rows containing non-finite values (stat_boxplot). 
Warning messages:
1: Removed 10169 rows containing non-finite values (stat_boxplot). 
2: Removed 10169 rows containing non-finite values (stat_boxplot). 
3: Removed 1 rows containing missing values (geom_hline). 


##### Group according to enrichment score and overlapped ratio (>=10%)

In [82]:
%%bash
dirPATH=/mnt/Storage/home/yanghui/imprinting/result.2021/Exploring/CHMsInOtherContexts/CellStateTransition/CHMOrganization/Universal_specific/Features/Repeats
for repeatType in LTR LINE SINE
do
    cd ${dirPATH}/Universal_ProcessSpecific_NonCHMsCpGrich/${repeatType}_Subfamily
    awk 'BEGIN{FS=OFS="\t"}{if($1!="xValue" && $1>=10 && $2>=1){print $5;}}' enrichDetails.ggplot_scatter.txt | sort -u > subfml_Enriched_esor.txt #
    grep -vwf subfml_Enriched.txt all.subfml.txt > subfml_NonEnriched_esor.txt #
done # for repeatType end

In [83]:
%%bash
anPATH=/mnt/Storage/home/yanghui/annotations
dirPATH=/mnt/Storage/home/yanghui/imprinting/result.2021/Exploring/CHMsInOtherContexts/CellStateTransition/CHMOrganization/Universal_specific/Features/Repeats

for repeatType in LTR LINE SINE
do
    cd ${dirPATH}/Universal_ProcessSpecific_NonCHMsCpGrich/${repeatType}_Subfamily
    for group in Enriched NonEnriched
    do
        if [ -s subfml_${group}_esor.txt ]
        then
            for subfml in $(cut -f 1 subfml_${group}_esor.txt)
            do
                cat ${anPATH}/mm10/Repeats/${repeatType}/*/mm10.${subfml}.bed | \
                    intersectBed -a - -b ${anPATH}/mm10/mm10_euch.chrom.limits | \
                    cut -f 1-3 >> subfml_${group}_esor.bed
            done # for subfml end
        fi
        sort -k1,1 -k2,2n -u subfml_${group}_esor.bed > subfml_${group}_esor.bed.tmp && mv subfml_${group}_esor.bed.tmp subfml_${group}_esor.bed
    done # for group end
done # for repeatType end

sort: cannot read: subfml_Enriched_esor.bed: No such file or directory
sort: cannot read: subfml_Enriched_esor.bed: No such file or directory


In [84]:
%%bash
dirPATH=/mnt/Storage/home/yanghui/imprinting/result.2021/Exploring/CHMsInOtherContexts/CellStateTransition/CHMOrganization/Universal_specific/Features/Repeats
anPATH=/mnt/Storage/home/yanghui/annotations
shPATH=/mnt/Storage/home/yanghui/scripts/shell
rPATH=/mnt/Storage/home/yanghui/scripts/R

for repeatType in LTR
do
    cd ${dirPATH}/Universal_ProcessSpecific_NonCHMsCpGrich/${repeatType}_Subfamily
    echo -e "xValue\tyValue\tgroup" > phastCons60wayPlacental_subfml_esor.ggplot_basic.txt
    for group in Enriched NonEnriched
    do
        if [ -f subfml_${group}_esor.bed ]
        then
            bash ${shPATH}/averageSignalInRegion.sh \
                subfml_${group}_esor.bed \
                ${anPATH}/mm10/Conservation/mm10.60way.phastCons60wayPlacental.bw \
                1 \
                subfml_${group}_esor.phastConsPlacental.txt
            
            awk -v XVALUE=${group} -v GROUP=${repeatType} 'BEGIN{FS=OFS="\t"}{print XVALUE, $4, GROUP;}' \
                subfml_${group}_esor.phastConsPlacental.txt >> phastCons60wayPlacental_subfml_esor.ggplot_basic.txt
        fi
    done # for group end
    
    
    # Violin plot
    Rscript ${rPATH}/ggplot_violin.r \
        phastCons60wayPlacental_subfml_esor.ggplot_basic.txt \
        phastCons60wayPlacental_subfml_esor.ggplot_violin.pdf \
        ${repeatType} \
        "" \
        "Conservation score" \
        0 \
        0.6 \
        0.2 \
        "Enriched;NonEnriched" \
        "${repeatType}" \
        3 \
        4
                
    # box plot
    Rscript ${rPATH}/ggplot_boxplot.r \
        phastCons60wayPlacental_subfml_esor.ggplot_basic.txt \
        phastCons60wayPlacental_subfml_esor.ggplot_boxplot.pdf \
        ${repeatType} \
        "" \
        "Conservation score" \
        0 \
        0.6 \
        0.2 \
        "Enriched;NonEnriched" \
        "${repeatType}" \
        3 \
        4
done # for repeatType end

null device 
          1 
null device 
          1 
[1] "wilcox.test of significance between different xValues"
# A tibble: 1 × 8
  .y.    group1   group2             p   p.adj p.format p.signif method  
  <chr>  <chr>    <chr>          <dbl>   <dbl> <chr>    <chr>    <chr>   
1 yValue Enriched NonEnriched 3.06e-62 3.1e-62 <2e-16   ****     Wilcoxon
[1] "t.test of significance between different xValues"
# A tibble: 1 × 8
  .y.    group1   group2             p   p.adj p.format p.signif method
  <chr>  <chr>    <chr>          <dbl>   <dbl> <chr>    <chr>    <chr> 
1 yValue Enriched NonEnriched 1.78e-27 1.8e-27 <2e-16   ****     T-test


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



### Divide according to chromatin states

#### Obtain enrichment score

In [65]:
%%bash
dirPATH=/mnt/Storage/home/yanghui/imprinting/result.2021/Exploring/CHMsInOtherContexts/CellStateTransition/CHMOrganization/Universal_specific/Features/Repeats
shPATH=/mnt/Storage/home/yanghui/scripts/shell
cd ${dirPATH}/Universal_ProcessSpecific_NonCHMsCpGrich

for group in UniversalCHMs ProcessSpecificCHMs NonCHMsCpGrich
do
    for repeatType in LTR LINE SINE
    do
    	for cmpt in A B
    	do
        	bash ${shPATH}/regionEnrichment_${repeatType}_mm10_euch.sh \
        	    "${dirPATH}/Universal_ProcessSpecific_NonCHMsCpGrich/${repeatType}_Subfamily" \
        	    "${dirPATH}/Universal_ProcessSpecific_NonCHMsCpGrich/${group}.cmpt_${cmpt}.bed" \
        	    "${group}.cmpt_${cmpt}"
        done # for cmpt end
    done # for repeatType end
done # for group end

null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 

Runtime error (func=(main), adr=3): Divide by zero
Runtime error (func=(main), adr=3): Divide by zero
Runtime error (func=(main), adr=3): Divide by zero
Runtime error (func=(main), adr=3): Divide by zero
Runtime error (func=(main), adr=3): Divide by zero
Runtime error (func=(main), adr=3): Divide by zero


#### Sina plot

##### Prepare for sina plot

In [69]:
%%bash
dirPATH=/mnt/Storage/home/yanghui/imprinting/result.2021/Exploring/CHMsInOtherContexts/CellStateTransition/CHMOrganization/Universal_specific/Features/Repeats

for repeatType in LTR LINE SINE
do
    cd ${dirPATH}/Universal_ProcessSpecific_NonCHMsCpGrich/${repeatType}_Subfamily
    echo -e "xValue\tyValue\tgroup\tcol\tlabel" > enrichmentScore_cmpt.ggplot_sina.txt
    for group in UniversalCHMs ProcessSpecificCHMs NonCHMsCpGrich
    do
        for cmpt in A B
        do
            for fmlEnrich in $(ls ${group}.cmpt_${cmpt}.EnrichScore_${repeatType}_*.txt)
            do
                fmlName=$(echo ${fmlEnrich#*EnrichScore_${repeatType}_*})
                fmlName=$(echo ${fmlName%.txt})
                tail -n +2 ${fmlEnrich} | grep -v "inf" | \
                    awk -v XVALUE=${group} -v GROUP=${cmpt} -v COL=${fmlName} 'BEGIN{FS=OFS="\t"}{\
                    print XVALUE, $2, GROUP, COL, $1;}' >> enrichmentScore_cmpt.ggplot_sina.txt
            done # for fmlEnrich end
        done # for cmpt end
    done # for group end
done # for repeatType end

##### Sina plot

In [71]:
%%bash
dirPATH=/mnt/Storage/home/yanghui/imprinting/result.2021/Exploring/CHMsInOtherContexts/CellStateTransition/CHMOrganization/Universal_specific/Features/Repeats
rPATH=/mnt/Storage/home/yanghui/scripts/R

LTR_sina(){
    cd ${dirPATH}/Universal_ProcessSpecific_NonCHMsCpGrich/LTR_Subfamily
    Rscript ${rPATH}/ggplot_sina_colorfull.r \
        enrichmentScore_cmpt.ggplot_sina.txt \
        enrichmentScore_cmpt_filtered.ggplot_sina.pdf \
        "Enrichment score" \
        "-12;8;4" \
        "UniversalCHMs;ProcessSpecificCHMs;NonCHMsCpGrich" \
        "A;B" \
        "ERV1;ERVK;ERVL;ERVL-MaLR;Gypsy" \
        "#CE0013;#C7A609;#64C0AB;#A14C94;#1187CD" \
        "0.2" \
        "0.5" \
        "1" \
        "5" \
        "5"
}
LTR_sina

LINE_sina(){
    cd ${dirPATH}/Universal_ProcessSpecific_NonCHMsCpGrich/LINE_Subfamily
    Rscript ${rPATH}/ggplot_sina_colorfull.r \
        enrichmentScore_cmpt.ggplot_sina.txt \
        enrichmentScore_cmpt_filtered.ggplot_sina.pdf \
        "Enrichment score" \
        "-12;8;4" \
        "UniversalCHMs;ProcessSpecificCHMs;NonCHMsCpGrich" \
        "A;B" \
        "CR1;Dong-R4;Jockey;L1;L1-Tx1;L2;Penelope;RTE-BovB;RTE-X" \
        "#CE0013;#16557A;#C7A609;#87C232;#64C0AB;#A14C94;#8B7E75;#1187CD;#000000" \
        "0.2" \
        "0.5" \
        "1" \
        "5" \
        "5"
}
LINE_sina

SINE_sina(){
    cd ${dirPATH}/Universal_ProcessSpecific_NonCHMsCpGrich/SINE_Subfamily
    Rscript ${rPATH}/ggplot_sina_colorfull.r \
        enrichmentScore_cmpt.ggplot_sina.txt \
        enrichmentScore_cmpt_filtered.ggplot_sina.pdf \
        "Enrichment score" \
        "-12;8;4" \
        "UniversalCHMs;ProcessSpecificCHMs;NonCHMsCpGrich" \
        "A;B" \
        "5S-Deu-L2;Alu;B2;B4;ID;MIR;tRNA;tRNA-Deu;tRNA-RTE" \
        "#CE0013;#16557A;#C7A609;#87C232;#64C0AB;#A14C94;#8B7E75;#1187CD;#000000" \
        "0.2" \
        "0.5" \
        "1" \
        "5" \
        "5"
}
SINE_sina

# R: Can't see significant difference between A and B compartment through this methods

[1] "#CE0013" "#C7A609" "#64C0AB" "#A14C94" "#1187CD"
[1] "ERV1"      "ERVK"      "ERVL"      "ERVL-MaLR" "Gypsy"    

  1   2   3   4   5 
182 594 114 230   9 
null device 
          1 
[1] "#CE0013" "#16557A" "#C7A609" "#87C232" "#64C0AB" "#A14C94" "#8B7E75"
[8] "#1187CD" "#000000"
[1] "CR1"      "Dong-R4"  "Jockey"   "L1"       "L1-Tx1"   "L2"       "Penelope"
[8] "RTE-BovB" "RTE-X"   

  1   3   4   6   7   8   9 
 16   1 396  34   2   3   3 
null device 
          1 
[1] "#CE0013" "#16557A" "#C7A609" "#87C232" "#64C0AB" "#A14C94" "#8B7E75"
[8] "#1187CD" "#000000"
[1] "5S-Deu-L2" "Alu"       "B2"        "B4"        "ID"        "MIR"      
[7] "tRNA"      "tRNA-Deu"  "tRNA-RTE" 

 1  2  3  4  5  6  7  8  9 
 1 80 28 24 16 23  2  1  3 
null device 
          1 


##### Prepare sina to show A / B as corresponding xValue

In [74]:
%%bash
dirPATH=/mnt/Storage/home/yanghui/imprinting/result.2021/Exploring/CHMsInOtherContexts/CellStateTransition/CHMOrganization/Universal_specific/Features/Repeats

for repeatType in LTR LINE SINE
do
    cd ${dirPATH}/Universal_ProcessSpecific_NonCHMsCpGrich/${repeatType}_Subfamily
    echo -e "xValue\tyValue\tgroup\tcol\tlabel" > enrichmentScore_cmpt_crsp.ggplot_sina.txt
    for group in UniversalCHMs ProcessSpecificCHMs NonCHMsCpGrich
    do
        for cmpt in A B
        do
            for fmlEnrich in $(ls ${group}.cmpt_${cmpt}.EnrichScore_${repeatType}_*.txt)
            do
                fmlName=$(echo ${fmlEnrich#*EnrichScore_${repeatType}_*})
                fmlName=$(echo ${fmlName%.txt})
                tail -n +2 ${fmlEnrich} | grep -v "inf" | \
                    awk -v XVALUE=${group}.${cmpt} -v GROUP=${repeatType} -v COL=${fmlName} 'BEGIN{FS=OFS="\t"}{\
                    print XVALUE, $2, GROUP, COL, $1;}' >> enrichmentScore_cmpt_crsp.ggplot_sina.txt
            done # for fmlEnrich end
        done # for cmpt end
    done # for group end
done # for repeatType end

In [75]:
%%bash
dirPATH=/mnt/Storage/home/yanghui/imprinting/result.2021/Exploring/CHMsInOtherContexts/CellStateTransition/CHMOrganization/Universal_specific/Features/Repeats
rPATH=/mnt/Storage/home/yanghui/scripts/R

LTR_sina(){
    cd ${dirPATH}/Universal_ProcessSpecific_NonCHMsCpGrich/LTR_Subfamily
    Rscript ${rPATH}/ggplot_sina_colorfull.r \
        enrichmentScore_cmpt_crsp.ggplot_sina.txt \
        enrichmentScore_cmpt_crsp_filtered.ggplot_sina.pdf \
        "Enrichment score" \
        "-12;8;4" \
        "UniversalCHMs.A;UniversalCHMs.B;ProcessSpecificCHMs.A;ProcessSpecificCHMs.B;NonCHMsCpGrich.A;NonCHMsCpGrich.B" \
        "LTR" \
        "ERV1;ERVK;ERVL;ERVL-MaLR;Gypsy" \
        "#CE0013;#C7A609;#64C0AB;#A14C94;#1187CD" \
        "0.2" \
        "0.5" \
        "1" \
        "5" \
        "5"
}
LTR_sina

LINE_sina(){
    cd ${dirPATH}/Universal_ProcessSpecific_NonCHMsCpGrich/LINE_Subfamily
    Rscript ${rPATH}/ggplot_sina_colorfull.r \
        enrichmentScore_cmpt_crsp.ggplot_sina.txt \
        enrichmentScore_cmpt_crsp_filtered.ggplot_sina.pdf \
        "Enrichment score" \
        "-12;8;4" \
        "UniversalCHMs.A;UniversalCHMs.B;ProcessSpecificCHMs.A;ProcessSpecificCHMs.B;NonCHMsCpGrich.A;NonCHMsCpGrich.B" \
        "LINE" \
        "CR1;Dong-R4;Jockey;L1;L1-Tx1;L2;Penelope;RTE-BovB;RTE-X" \
        "#CE0013;#16557A;#C7A609;#87C232;#64C0AB;#A14C94;#8B7E75;#1187CD;#000000" \
        "0.2" \
        "0.5" \
        "1" \
        "5" \
        "5"
}
LINE_sina

SINE_sina(){
    cd ${dirPATH}/Universal_ProcessSpecific_NonCHMsCpGrich/SINE_Subfamily
    Rscript ${rPATH}/ggplot_sina_colorfull.r \
        enrichmentScore_cmpt_crsp.ggplot_sina.txt \
        enrichmentScore_cmpt_crsp_filtered.ggplot_sina.pdf \
        "Enrichment score" \
        "-12;8;4" \
        "UniversalCHMs.A;UniversalCHMs.B;ProcessSpecificCHMs.A;ProcessSpecificCHMs.B;NonCHMsCpGrich.A;NonCHMsCpGrich.B" \
        "SINE" \
        "5S-Deu-L2;Alu;B2;B4;ID;MIR;tRNA;tRNA-Deu;tRNA-RTE" \
        "#CE0013;#16557A;#C7A609;#87C232;#64C0AB;#A14C94;#8B7E75;#1187CD;#000000" \
        "0.2" \
        "0.5" \
        "1" \
        "5" \
        "5"
}
SINE_sina

[1] "#CE0013" "#C7A609" "#64C0AB" "#A14C94" "#1187CD"
[1] "ERV1"      "ERVK"      "ERVL"      "ERVL-MaLR" "Gypsy"    

  1   2   3   4   5 
182 594 114 230   9 
null device 
          1 
[1] "#CE0013" "#16557A" "#C7A609" "#87C232" "#64C0AB" "#A14C94" "#8B7E75"
[8] "#1187CD" "#000000"
[1] "CR1"      "Dong-R4"  "Jockey"   "L1"       "L1-Tx1"   "L2"       "Penelope"
[8] "RTE-BovB" "RTE-X"   

  1   3   4   6   7   8   9 
 16   1 396  34   2   3   3 
null device 
          1 
[1] "#CE0013" "#16557A" "#C7A609" "#87C232" "#64C0AB" "#A14C94" "#8B7E75"
[8] "#1187CD" "#000000"
[1] "5S-Deu-L2" "Alu"       "B2"        "B4"        "ID"        "MIR"      
[7] "tRNA"      "tRNA-Deu"  "tRNA-RTE" 

 1  2  3  4  5  6  7  8  9 
 1 80 28 24 16 23  2  1  3 
null device 
          1 


#### Scatter plot

In [77]:
%%bash
dirPATH=/mnt/Storage/home/yanghui/imprinting/result.2021/Exploring/CHMsInOtherContexts/CellStateTransition/CHMOrganization/Universal_specific/Features/Repeats
rPATH=/mnt/Storage/home/yanghui/scripts/R

LTR_F(){
    cd ${dirPATH}/Universal_ProcessSpecific_NonCHMsCpGrich/LTR_Subfamily
    echo -e "xValue\tyValue\tgroup\tshape\tsubfamily" > enrichDetails_cmpt.ggplot_scatter.txt # group: ERV1...; shape: UniversalCHMs
    for group in UniversalCHMs ProcessSpecificCHMs NonCHMsCpGrich
    do
        for cmpt in A B
        do
            for fml in ERV1 ERVK ERVL ERVL-MaLR Gypsy
            do
                paste ${group}.cmpt_${cmpt}.overlaped_LTR_${fml}.txt ${group}.cmpt_${cmpt}.EnrichScore_LTR_${fml}.txt | \
                    tail -n +2 | awk -v GROUP=${fml} -v SHAPE=${group}.cmpt_${cmpt} 'BEGIN{FS=OFS="\t"}{if($1==$4 && $5!="-inf"){print $3*100, $5, GROUP, SHAPE, $1;}}' >> enrichDetails_cmpt.ggplot_scatter.txt
            done # for fml end
        done # for cmpt end
    done # for group end
    
    Rscript ${rPATH}/ggplot_scatterplot_basic.r \
        enrichDetails_cmpt.ggplot_scatter.txt \
        enrichDetails_cmpt \
        "Overlapped ratio (%)" \
        "Enrichment score" \
        "0;100;20" \
        "-12;8;4" \
        "ERV1;ERVK;ERVL;ERVL-MaLR;Gypsy" \
        "T;1;0" \
        "0.75" \
        "5" \
        "5" \
        "Set1"
}
LTR_F

LINE_F(){
    cd ${dirPATH}/Universal_ProcessSpecific_NonCHMsCpGrich/LINE_Subfamily
    echo -e "xValue\tyValue\tgroup\tshape\tsubfamily" > enrichDetails_cmpt.ggplot_scatter.txt # group: ERV1...; shape: UniversalCHMs
    for group in UniversalCHMs ProcessSpecificCHMs NonCHMsCpGrich
    do
        for cmpt in A B
        do
            for fml in CR1 Dong-R4 Jockey L1 L1-Tx1 L2 Penelope RTE-BovB RTE-X
            do
                paste ${group}.cmpt_${cmpt}.overlaped_LINE_${fml}.txt ${group}.cmpt_${cmpt}.EnrichScore_LINE_${fml}.txt | \
                    tail -n +2 | awk -v GROUP=${fml} -v SHAPE=${group}.cmpt_${cmpt} 'BEGIN{FS=OFS="\t"}{if($1==$4 && $5!="-inf"){print $3*100, $5, GROUP, SHAPE, $1;}}' >> enrichDetails_cmpt.ggplot_scatter.txt
            done # for fml end
        done # for cmpt end
    done # for group end
    
    Rscript ${rPATH}/ggplot_scatterplot_basic.r \
        enrichDetails_cmpt.ggplot_scatter.txt \
        enrichDetails_cmpt \
        "Overlapped ratio (%)" \
        "Enrichment score" \
        "0;100;20" \
        "-12;8;4" \
        "CR1;Dong-R4;Jockey;L1;L1-Tx1;L2;Penelope;RTE-BovB;RTE-X" \
        "T;1;0" \
        "0.75" \
        "5" \
        "5" \
        "Set1"
}
LINE_F

SINE_F(){
    cd ${dirPATH}/Universal_ProcessSpecific_NonCHMsCpGrich/SINE_Subfamily
    echo -e "xValue\tyValue\tgroup\tshape\tsubfamily" > enrichDetails_cmpt.ggplot_scatter.txt # group: ERV1...; shape: UniversalCHMs
    for group in UniversalCHMs ProcessSpecificCHMs NonCHMsCpGrich
    do
        for cmpt in A B
        do
            for fml in 5S-Deu-L2 Alu B2 B4 ID MIR tRNA tRNA-Deu tRNA-RTE
            do
                paste ${group}.cmpt_${cmpt}.overlaped_SINE_${fml}.txt ${group}.cmpt_${cmpt}.EnrichScore_SINE_${fml}.txt | \
                    tail -n +2 | awk -v GROUP=${fml} -v SHAPE=${group}.cmpt_${cmpt} 'BEGIN{FS=OFS="\t"}{if($1==$4 && $5!="-inf"){print $3*100, $5, GROUP, SHAPE, $1;}}' >> enrichDetails_cmpt.ggplot_scatter.txt
            done # for fml end
        done # for cmpt end
    done # for group end
    
    Rscript ${rPATH}/ggplot_scatterplot_basic.r \
        enrichDetails_cmpt.ggplot_scatter.txt \
        enrichDetails_cmpt \
        "Overlapped ratio (%)" \
        "Enrichment score" \
        "0;100;20" \
        "-12;8;4" \
        "5S-Deu-L2;Alu;B2;B4;ID;MIR;tRNA;tRNA-Deu;tRNA-RTE" \
        "T;1;0" \
        "0.75" \
        "5" \
        "5" \
        "Set1"
}
SINE_F

null device 
          1 
null device 
          1 
null device 
          1 


#### Enriched vs Non-enriched subfamilies

In [ ]:
%%bash
dirPATH=/mnt/Storage/home/yanghui/imprinting/result.2021/Exploring/CHMsInOtherContexts/CellStateTransition/CHMOrganization/Universal_specific/Features/Repeats

for repeatType in LTR LINE SINE
do
    cd ${dirPATH}/Universal_ProcessSpecific_NonCHMsCpGrich/${repeatType}_Subfamily
    for group in UniversalCHMs ProcessSpecificCHMs NonCHMsCpGrich
    do
        echo -e "value\tgroup" > ${group}.enriched.ggplot_pieplot.txt
        for fmlEnrich in $(ls ${group}.EnrichScore_${repeatType}_*.txt)
        do
            fmlName=$(echo ${fmlEnrich#*EnrichScore_${repeatType}_*})
            fmlName=$(echo ${fmlName%.txt})
            tail -n +2 ${fmlEnrich} | \
                awk -v GROUP=${fmlName} 'BEGIN{FS=OFS="\t";TOTAL=0;}{\
                if($2>=1){TOTAL+=1;}}END{print TOTAL, GROUP;}' >> ${group}.enriched.ggplot_pieplot.txt
        done # for fmlEnrich end
    done # for group end
done # for repeatType end

# Universal vs NonCHM CpGrich

## Region enrichment

### Don't divide according to chromatin states

In [39]:
%%bash
dirPATH=/mnt/Storage/home/yanghui/imprinting/result.2021/Exploring/CHMsInOtherContexts/CellStateTransition/CHMOrganization/Universal_specific/Features/Repeats
rPATH=/mnt/Storage/home/yanghui/scripts/R
mkdir -p ${dirPATH}/UniversalCHMs_NonCHMsCpGrich;cd ${dirPATH}/UniversalCHMs_NonCHMsCpGrich
Rscript ${rPATH}/ggplot_barplot.r \
    "${dirPATH}/Universal_ProcessSpecific_NonCHMsCpGrich/regionEnrich.ggplot_basic.txt" \
    "regionEnrich.ggplot_barplot.pdf" \
    "Region enrichment" \
    "" \
    "Enrichment score" \
    "-6;6.2;2" \
    "CGI;promoter;geneBody;exon;intron;LTR;LINE;SINE" \
    "UniversalCHMs;NonCHMsCpGrich" \
    "dodge" \
    "7" \
    "3" \
    "1" \
    "F"

null device 
          1 


### Divide according to chromatin states

In [41]:
%%bash
dirPATH=/mnt/Storage/home/yanghui/imprinting/result.2021/Exploring/CHMsInOtherContexts/CellStateTransition/CHMOrganization/Universal_specific/Features/Repeats
rPATH=/mnt/Storage/home/yanghui/scripts/R
mkdir -p ${dirPATH}/UniversalCHMs_NonCHMsCpGrich;cd ${dirPATH}/UniversalCHMs_NonCHMsCpGrich

Rscript ${rPATH}/ggplot_barplot.r \
    "${dirPATH}/Universal_ProcessSpecific_NonCHMsCpGrich/regionEnrich_cmpt.ggplot_basic.txt" \
    "regionEnrich_cmpt.ggplot_barplot.pdf" \
    "Region enrichment" \
    "" \
    "Enrichment score" \
    "-6;6.2;2" \
    "CGI;promoter;geneBody;exon;intron;LTR;LINE;SINE" \
    "UniversalCHMs ov A;UniversalCHMs ov B;NonCHMsCpGrich ov A;NonCHMsCpGrich ov B" \
    "dodge" \
    "7" \
    "3" \
    "1" \
    "F"

null device 
          1 


## Repeats subfamily enrichment

### Sina plot

In [58]:
%%bash
dirPATH=/mnt/Storage/home/yanghui/imprinting/result.2021/Exploring/CHMsInOtherContexts/CellStateTransition/CHMOrganization/Universal_specific/Features/Repeats
rPATH=/mnt/Storage/home/yanghui/scripts/R

LTR_sina(){
    mkdir -p ${dirPATH}/UniversalCHMs_NonCHMsCpGrich/LTR_Subfamily;cd ${dirPATH}/UniversalCHMs_NonCHMsCpGrich/LTR_Subfamily
    Rscript ${rPATH}/ggplot_sina_colorfull.r \
        ${dirPATH}/Universal_ProcessSpecific_NonCHMsCpGrich/LTR_Subfamily/enrichmentScore.ggplot_sina.txt \
        enrichmentScore_filtered.ggplot_sina.pdf \
        "Enrichment score" \
        "-12;8;4" \
        "UniversalCHMs;NonCHMsCpGrich" \
        "LTR" \
        "ERV1;ERVK;ERVL;ERVL-MaLR;Gypsy" \
        "#CE0013;#C7A609;#64C0AB;#A14C94;#1187CD" \
        "0.2" \
        "0.75" \
        "1" \
        "6" \
        "6.5"
}
LTR_sina

LINE_sina(){
    mkdir -p ${dirPATH}/UniversalCHMs_NonCHMsCpGrich/LINE_Subfamily;cd ${dirPATH}/UniversalCHMs_NonCHMsCpGrich/LINE_Subfamily
    Rscript ${rPATH}/ggplot_sina_colorfull.r \
        ${dirPATH}/Universal_ProcessSpecific_NonCHMsCpGrich/LINE_Subfamily/enrichmentScore.ggplot_sina.txt \
        enrichmentScore_filtered.ggplot_sina.pdf \
        "Enrichment score" \
        "-12;8;4" \
        "UniversalCHMs;NonCHMsCpGrich" \
        "LINE" \
        "CR1;Dong-R4;Jockey;L1;L1-Tx1;L2;Penelope;RTE-BovB;RTE-X" \
        "#CE0013;#16557A;#C7A609;#87C232;#64C0AB;#A14C94;#8B7E75;#1187CD;#000000" \
        "0.2" \
        "0.75" \
        "1" \
        "6" \
        "6.2"
}
LINE_sina

SINE_sina(){
    mkdir -p ${dirPATH}/UniversalCHMs_NonCHMsCpGrich/SINE_Subfamily;cd ${dirPATH}/UniversalCHMs_NonCHMsCpGrich/SINE_Subfamily
    Rscript ${rPATH}/ggplot_sina_colorfull.r \
        ${dirPATH}/Universal_ProcessSpecific_NonCHMsCpGrich/SINE_Subfamily/enrichmentScore.ggplot_sina.txt \
        enrichmentScore_filtered.ggplot_sina.pdf \
        "Enrichment score" \
        "-12;8;4" \
        "UniversalCHMs;NonCHMsCpGrich" \
        "SINE" \
        "5S-Deu-L2;Alu;B2;B4;ID;MIR;tRNA;tRNA-Deu;tRNA-RTE" \
        "#CE0013;#16557A;#C7A609;#87C232;#64C0AB;#A14C94;#8B7E75;#1187CD;#000000" \
        "0.2" \
        "0.75" \
        "1" \
        "6" \
        "6.1"
}
SINE_sina

[1] "#CE0013" "#C7A609" "#64C0AB" "#A14C94" "#1187CD"
[1] "ERV1"      "ERVK"      "ERVL"      "ERVL-MaLR" "Gypsy"    

  1   2   3   4   5 
 91 255  69 101   8 
null device 
          1 
[1] "#CE0013" "#16557A" "#C7A609" "#87C232" "#64C0AB" "#A14C94" "#8B7E75"
[8] "#1187CD" "#000000"
[1] "CR1"      "Dong-R4"  "Jockey"   "L1"       "L1-Tx1"   "L2"       "Penelope"
[8] "RTE-BovB" "RTE-X"   

  1   4   6   7   8   9 
 12 181  15   2   2   3 
null device 
          1 
[1] "#CE0013" "#16557A" "#C7A609" "#87C232" "#64C0AB" "#A14C94" "#8B7E75"
[8] "#1187CD" "#000000"
[1] "5S-Deu-L2" "Alu"       "B2"        "B4"        "ID"        "MIR"      
[7] "tRNA"      "tRNA-Deu"  "tRNA-RTE" 

 1  2  3  4  5  6  7  8  9 
 1 31 10  8  6 10  1  1  2 
null device 
          1 


### Scatter plot

In [64]:
%%bash
dirPATH=/mnt/Storage/home/yanghui/imprinting/result.2021/Exploring/CHMsInOtherContexts/CellStateTransition/CHMOrganization/Universal_specific/Features/Repeats
rPATH=/mnt/Storage/home/yanghui/scripts/R

LTR_F(){
    cd ${dirPATH}/UniversalCHMs_NonCHMsCpGrich/LTR_Subfamily
    # echo -e "xValue\tyValue\tgroup\tshape\tsubfamily" > enrichDetails.ggplot_scatter.txt # group: ERV1...; shape: UniversalCHMs
    # for group in UniversalCHMs NonCHMsCpGrich
    # do
    #     for fml in ERV1 ERVK ERVL ERVL-MaLR Gypsy
    #     do
    #         paste ${dirPATH}/Universal_ProcessSpecific_NonCHMsCpGrich/LTR_Subfamily/${group}.overlaped_LTR_${fml}.txt ${dirPATH}/Universal_ProcessSpecific_NonCHMsCpGrich/LTR_Subfamily/${group}.EnrichScore_LTR_${fml}.txt | \
    #             tail -n +2 | awk -v GROUP=${fml} -v SHAPE=${group} 'BEGIN{FS=OFS="\t"}{if($1==$4 && $5!="-inf"){print $3*100, $5, GROUP, SHAPE, $1;}}' >> enrichDetails.ggplot_scatter.txt
    #     done # for fml end
    # done # for group end
    
    
    
    Rscript ${rPATH}/ggplot_scatterplot_basic.r \
        enrichDetails.ggplot_scatter.txt \
        enrichDetails \
        "Overlapped ratio (%)" \
        "Enrichment score" \
        "0;80;20" \
        "-12;8;4" \
        "ERV1;ERVK;ERVL;ERVL-MaLR;Gypsy" \
        "T;1;0" \
        "0.75" \
        "5" \
        "5" \
        "Set1"
}
LTR_F

LINE_F(){
    cd ${dirPATH}/UniversalCHMs_NonCHMsCpGrich/LINE_Subfamily
    # echo -e "xValue\tyValue\tgroup\tshape\tsubfamily" > enrichDetails.ggplot_scatter.txt # group: ERV1...; shape: UniversalCHMs
    # for group in UniversalCHMs NonCHMsCpGrich
    # do
    #     for fml in CR1 Dong-R4 Jockey L1 L1-Tx1 L2 Penelope RTE-BovB RTE-X
    #     do
    #         paste ${dirPATH}/Universal_ProcessSpecific_NonCHMsCpGrich/LINE_Subfamily/${group}.overlaped_LINE_${fml}.txt ${dirPATH}/Universal_ProcessSpecific_NonCHMsCpGrich/LINE_Subfamily/${group}.EnrichScore_LINE_${fml}.txt | \
    #             tail -n +2 | awk -v GROUP=${fml} -v SHAPE=${group} 'BEGIN{FS=OFS="\t"}{if($1==$4 && $5!="-inf"){print $3*100, $5, GROUP, SHAPE, $1;}}' >> enrichDetails.ggplot_scatter.txt
    #     done # for fml end
    # done # for group end
    
    
    
    Rscript ${rPATH}/ggplot_scatterplot_basic.r \
        enrichDetails.ggplot_scatter.txt \
        enrichDetails \
        "Overlapped ratio (%)" \
        "Enrichment score" \
        "0;100;20" \
        "-12;8;4" \
        "CR1;Dong-R4;Jockey;L1;L1-Tx1;L2;Penelope;RTE-BovB;RTE-X" \
        "T;1;0" \
        "0.75" \
        "5" \
        "5" \
        "Set1"
}
LINE_F

SINE_F(){
    cd ${dirPATH}/UniversalCHMs_NonCHMsCpGrich/SINE_Subfamily
    # echo -e "xValue\tyValue\tgroup\tshape\tsubfamily" > enrichDetails.ggplot_scatter.txt # group: ERV1...; shape: UniversalCHMs
    # for group in UniversalCHMs NonCHMsCpGrich
    # do
    #     for fml in 5S-Deu-L2 Alu B2 B4 ID MIR tRNA tRNA-Deu tRNA-RTE
    #     do
    #         paste ${dirPATH}/Universal_ProcessSpecific_NonCHMsCpGrich/SINE_Subfamily/${group}.overlaped_SINE_${fml}.txt ${dirPATH}/Universal_ProcessSpecific_NonCHMsCpGrich/SINE_Subfamily/${group}.EnrichScore_SINE_${fml}.txt | \
    #             tail -n +2 | awk -v GROUP=${fml} -v SHAPE=${group} 'BEGIN{FS=OFS="\t"}{if($1==$4 && $5!="-inf"){print $3*100, $5, GROUP, SHAPE, $1;}}' >> enrichDetails.ggplot_scatter.txt
    #     done # for fml end
    # done # for group end
    
    
    
    Rscript ${rPATH}/ggplot_scatterplot_basic.r \
        enrichDetails.ggplot_scatter.txt \
        enrichDetails \
        "Overlapped ratio (%)" \
        "Enrichment score" \
        "0;100;20" \
        "-12;8;4" \
        "5S-Deu-L2;Alu;B2;B4;ID;MIR;tRNA;tRNA-Deu;tRNA-RTE" \
        "T;1;0" \
        "0.75" \
        "5" \
        "5" \
        "Set1"
}
SINE_F

null device 
          1 
null device 
          1 
null device 
          1 


### Pie plot showing enriched subfamily

In [49]:
%%bash
dirPATH=/mnt/Storage/home/yanghui/imprinting/result.2021/Exploring/CHMsInOtherContexts/CellStateTransition/CHMOrganization/Universal_specific/Features/Repeats
rPATH=/mnt/Storage/home/yanghui/scripts/R

LTR_pie(){
    cd ${dirPATH}/UniversalCHMs_NonCHMsCpGrich/LTR_Subfamily
    for group in UniversalCHMs NonCHMsCpGrich
    do
        Rscript ${rPATH}/ggplot_pie.r \
        ${dirPATH}/Universal_ProcessSpecific_NonCHMsCpGrich/LTR_Subfamily/${group}.enriched.ggplot_pieplot.txt \
        ${group} \
        "ERV1;ERVK;ERVL;ERVL-MaLR;Gypsy" \
        "#CE0013;#C7A609;#64C0AB;#A14C94;#1187CD" \
        T \
        3.5 \
        3.5
    done # for group end
}
LTR_pie

LINE_pie(){
    cd ${dirPATH}/UniversalCHMs_NonCHMsCpGrich/LINE_Subfamily
    for group in UniversalCHMs NonCHMsCpGrich
    do
        Rscript ${rPATH}/ggplot_pie.r \
        ${dirPATH}/Universal_ProcessSpecific_NonCHMsCpGrich/LINE_Subfamily/${group}.enriched.ggplot_pieplot.txt \
        ${group} \
        "CR1;Dong-R4;Jockey;L1;L1-Tx1;L2;Penelope;RTE-BovB;RTE-X" \
        "#CE0013;#16557A;#C7A609;#87C232;#64C0AB;#A14C94;#8B7E75;#1187CD;#000000" \
        T \
        3.5 \
        3.5
    done # for group end
}
LINE_pie

SINE_pie(){
    cd ${dirPATH}/UniversalCHMs_NonCHMsCpGrich/SINE_Subfamily
    for group in UniversalCHMs NonCHMsCpGrich
    do
        Rscript ${rPATH}/ggplot_pie.r \
        ${dirPATH}/Universal_ProcessSpecific_NonCHMsCpGrich/SINE_Subfamily/${group}.enriched.ggplot_pieplot.txt \
        ${group} \
        "5S-Deu-L2;Alu;B2;B4;ID;MIR;tRNA;tRNA-Deu;tRNA-RTE" \
        "#CE0013;#16557A;#C7A609;#87C232;#64C0AB;#A14C94;#8B7E75;#1187CD;#000000" \
        T \
        3.5 \
        3.5
    done # for group end
}
SINE_pie

null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 


Warning messages:
1: Removed 9 rows containing missing values (position_stack). 
2: Removed 9 rows containing missing values (geom_text). 


### Conservation score

#### Obtain Enriched vs NonEnriched repeat subfamilies

In [51]:
%%bash
dirPATH=/mnt/Storage/home/yanghui/imprinting/result.2021/Exploring/CHMsInOtherContexts/CellStateTransition/CHMOrganization/Universal_specific/Features/Repeats
for repeatType in LTR LINE SINE
do
    cd ${dirPATH}/UniversalCHMs_NonCHMsCpGrich/${repeatType}_Subfamily
    sort -u ${dirPATH}/Universal_ProcessSpecific_NonCHMsCpGrich/${repeatType}_Subfamily/UniversalCHMs.subfml_enriched.txt > subfml_Enriched.txt #
    grep -vwf subfml_Enriched.txt ${dirPATH}/Universal_ProcessSpecific_NonCHMsCpGrich/${repeatType}_Subfamily/all.subfml.txt > subfml_NonEnriched.txt #
done # for repeatType end

#### Obtain corresponding annotation

In [52]:
%%bash
anPATH=/mnt/Storage/home/yanghui/annotations
dirPATH=/mnt/Storage/home/yanghui/imprinting/result.2021/Exploring/CHMsInOtherContexts/CellStateTransition/CHMOrganization/Universal_specific/Features/Repeats

for repeatType in LTR LINE SINE
do
    cd ${dirPATH}/UniversalCHMs_NonCHMsCpGrich/${repeatType}_Subfamily
    for group in Enriched NonEnriched
    do
        if [ -s subfml_${group}.txt ]
        then
            for subfml in $(cut -f 1 subfml_${group}.txt)
            do
                cat ${anPATH}/mm10/Repeats/${repeatType}/*/mm10.${subfml}.bed | \
                    intersectBed -a - -b ${anPATH}/mm10/mm10_euch.chrom.limits | \
                    cut -f 1-3 >> subfml_${group}.bed
            done # for subfml end
        fi
        sort -k1,1 -k2,2n -u subfml_${group}.bed > subfml_${group}.bed.tmp && mv subfml_${group}.bed.tmp subfml_${group}.bed
    done # for group end
done # for repeatType end

sort: cannot read: subfml_Enriched.bed: No such file or directory


#### Obtain average conservation score

In [54]:
%%bash
dirPATH=/mnt/Storage/home/yanghui/imprinting/result.2021/Exploring/CHMsInOtherContexts/CellStateTransition/CHMOrganization/Universal_specific/Features/Repeats
anPATH=/mnt/Storage/home/yanghui/annotations
shPATH=/mnt/Storage/home/yanghui/scripts/shell
rPATH=/mnt/Storage/home/yanghui/scripts/R

for repeatType in LTR LINE
do
    cd ${dirPATH}/UniversalCHMs_NonCHMsCpGrich/${repeatType}_Subfamily
    # echo -e "xValue\tyValue\tgroup" > phastCons60wayPlacental_subfml.ggplot_basic.txt
    # for group in Enriched NonEnriched
    # do
    #     if [ -f subfml_${group}.bed ]
    #     then
    #         bash ${shPATH}/averageSignalInRegion.sh \
    #             subfml_${group}.bed \
    #             ${anPATH}/mm10/Conservation/mm10.60way.phastCons60wayPlacental.bw \
    #             1 \
    #             subfml_${group}.phastConsPlacental.txt
    #         
    #         awk -v XVALUE=${group} -v GROUP=${repeatType} 'BEGIN{FS=OFS="\t"}{print XVALUE, $4, GROUP;}' \
    #             subfml_${group}.phastConsPlacental.txt >> phastCons60wayPlacental_subfml.ggplot_basic.txt
    #     fi
    # done # for group end
    
    # T: executed in 6m 8s, finished 17:09:18 2023-01-03
    
    
    # Violin plot
    Rscript ${rPATH}/ggplot_violin.r \
        phastCons60wayPlacental_subfml.ggplot_basic.txt \
        phastCons60wayPlacental_subfml.ggplot_violin.pdf \
        ${repeatType} \
        "" \
        "Conservation score" \
        0 \
        0.4 \
        0.2 \
        "Enriched;NonEnriched" \
        "${repeatType}" \
        3 \
        4
                
    # box plot
    Rscript ${rPATH}/ggplot_boxplot.r \
        phastCons60wayPlacental_subfml.ggplot_basic.txt \
        phastCons60wayPlacental_subfml.ggplot_boxplot.pdf \
        ${repeatType} \
        "" \
        "Conservation score" \
        0 \
        0.4 \
        0.2 \
        "Enriched;NonEnriched" \
        "${repeatType}" \
        3 \
        4
done # for repeatType end

null device 
          1 
null device 
          1 
[1] "wilcox.test of significance between different xValues"
# A tibble: 1 × 8
  .y.    group1   group2          p p.adj p.format p.signif method  
  <chr>  <chr>    <chr>       <dbl> <dbl> <chr>    <chr>    <chr>   
1 yValue Enriched NonEnriched     0     0 <2e-16   ****     Wilcoxon
[1] "t.test of significance between different xValues"
# A tibble: 1 × 8
  .y.    group1   group2              p    p.adj p.format p.signif method
  <chr>  <chr>    <chr>           <dbl>    <dbl> <chr>    <chr>    <chr> 
1 yValue Enriched NonEnriched 5.51e-158 5.5e-158 <2e-16   ****     T-test
null device 
          1 
null device 
          1 
[1] "wilcox.test of significance between different xValues"
# A tibble: 1 × 8
  .y.    group1   group2             p   p.adj p.format p.signif method  
  <chr>  <chr>    <chr>          <dbl>   <dbl> <chr>    <chr>    <chr>   
1 yValue Enriched NonEnriched 0.000323 0.00032 0.00032  ***      Wilcoxon
[1] "t.test of s

Warning messages:
1: Removed 24628 rows containing non-finite values (stat_ydensity). 
2: Removed 24628 rows containing non-finite values (stat_boxplot). 
Warning messages:
1: Removed 24628 rows containing non-finite values (stat_boxplot). 
2: Removed 24628 rows containing non-finite values (stat_boxplot). 
3: Removed 1 rows containing missing values (geom_hline). 
Warning messages:
1: Removed 28571 rows containing non-finite values (stat_ydensity). 
2: Removed 28571 rows containing non-finite values (stat_boxplot). 
Warning messages:
1: Removed 28571 rows containing non-finite values (stat_boxplot). 
2: Removed 28571 rows containing non-finite values (stat_boxplot). 
3: Removed 1 rows containing missing values (geom_hline). 


# Fig3 & FigS3

## Obtain annotation

In [86]:
%%bash
anPATH=/mnt/Storage/home/yanghui/annotations
dirPATH=/mnt/Storage/home/yanghui/imprinting/result.2021/Exploring/CHMsInOtherContexts/CellStateTransition/CHMOrganization/Universal_specific/Features/Repeats
dataPATH_ymw=/mnt/Storage/home/wangyiman/CHMsInOtherContexts/CellStateTransition/CHMOrganization
dataPATH_nonCHMsCpGrich1kb=/mnt/Storage/home/yanghui/imprinting/result.2021/Exploring/CHMsInOtherContexts/CellStateTransition/CHMOrganization/NonCHMsCpGrich.bed
mkdir -p ${dirPATH}/UniversalCHMs_NonCHMsCpGrich1kb;cd ${dirPATH}/UniversalCHMs_NonCHMsCpGrich1kb
cp ${dataPATH_ymw}/Universal_specific/Universal.CHM.bed UniversalCHMs.bed # 2,677
cp ${dataPATH_nonCHMsCpGrich1kb} NonCHMsCpGrich.bed # 22,069


### Updated annotation in compartment A and B

In [9]:
%%bash
anPATH=/mnt/Storage/home/yanghui/annotations
dirPATH=/mnt/Storage/home/yanghui/imprinting/result.2021/Exploring/CHMsInOtherContexts/CellStateTransition/CHMOrganization/Universal_specific/Features/Repeats
dataPATH_ymw_cpm=/mnt/Storage/home/wangyiman/CHMsInOtherContexts/CellStateTransition/CHMOrganization/Universal_specific/Features/ChromatinStates/Cavalli_Cell2017_ABcompartment_5tissue
mkdir -p ${dirPATH}/UniversalCHMs_NonCHMsCpGrich1kb;cd ${dirPATH}/UniversalCHMs_NonCHMsCpGrich1kb
cp ${dataPATH_ymw_cpm}/CHM_intersect_table/intersectOnly_UniversalCHM_5tissue_A.bed UniversalCHMs.cpm_A.bed # 553
cp ${dataPATH_ymw_cpm}/CHM_intersect_table/intersectOnly_UniversalCHM_5tissue_B.bed UniversalCHMs.cpm_B.bed # 946
cp ${dataPATH_ymw_cpm}/nonCHM_intersect_table/intersectOnly_nonCHM_5tissue_A.bed CpGrichNonCHMs.cpm_A.bed # 12,351
cp ${dataPATH_ymw_cpm}/nonCHM_intersect_table/intersectOnly_nonCHM_5tissue_B.bed CpGrichNonCHMs.cpm_B.bed # 639

## Region enrichment

### Don't divide according to chromatin state

#### Obtain region enrichment details

In [89]:
%%bash
dirPATH=/mnt/Storage/home/yanghui/imprinting/result.2021/Exploring/CHMsInOtherContexts/CellStateTransition/CHMOrganization/Universal_specific/Features/Repeats
anPATH=/mnt/Storage/home/yanghui/annotations
shPATH=/mnt/Storage/home/yanghui/scripts/shell
mkdir -p ${dirPATH}/UniversalCHMs_NonCHMsCpGrich1kb;cd ${dirPATH}/UniversalCHMs_NonCHMsCpGrich1kb

for group in UniversalCHMs NonCHMsCpGrich
do
    bash ${shPATH}/regionEnrichment_userDefined.sh \
        "${dirPATH}/UniversalCHMs_NonCHMsCpGrich1kb" \
        "mm10_euch" \
        "${anPATH}/mm10/mm10_euch.chrom.sizes" \
        "${group}.bed" \
        "${anPATH}/mm10/RegionEnrichment/mm10_euch.CGI_Unmasked.merged.bed;${anPATH}/mm10/RegionEnrichment/mm10_euch.promoter.merged.bed;${anPATH}/mm10/RegionEnrichment/mm10_euch.geneBody.merged.bed;${anPATH}/mm10/RegionEnrichment/mm10_euch.exon.merged.bed;${anPATH}/mm10/RegionEnrichment/mm10_euch.intron.merged.bed;${anPATH}/mm10/RegionEnrichment/mm10_euch.LTR.merged.bed;${anPATH}/mm10/RegionEnrichment/mm10_euch.LINE.merged.bed;${anPATH}/mm10/RegionEnrichment/mm10_euch.SINE.merged.bed" \
        "regionEnrich_${group}"
done # for group end

null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 


#### Plot together

In [90]:
%%bash
dirPATH=/mnt/Storage/home/yanghui/imprinting/result.2021/Exploring/CHMsInOtherContexts/CellStateTransition/CHMOrganization/Universal_specific/Features/Repeats
rPATH=/mnt/Storage/home/yanghui/scripts/R
cd ${dirPATH}/UniversalCHMs_NonCHMsCpGrich1kb

# Plot together
echo -e "xValue\tyValue\tgroup" > regionEnrich.ggplot_basic.txt
for group in UniversalCHMs NonCHMsCpGrich
do
    tail -n +2 regionEnrich_${group}.EnrichScore.txt | \
        awk -v GROUP=${group} 'BEGIN{FS=OFS="\t"}{\
        split($1, a, "_");\
        print a[1], $2, GROUP;}' >> regionEnrich.ggplot_basic.txt
done # for group end

Rscript ${rPATH}/ggplot_barplot.r \
    "regionEnrich.ggplot_basic.txt" \
    "regionEnrich.ggplot_barplot.pdf" \
    "Region enrichment" \
    "" \
    "Enrichment score" \
    "-6;6.2;2" \
    "CGI;promoter;geneBody;exon;intron;LTR;LINE;SINE" \
    "UniversalCHMs;NonCHMsCpGrich" \
    "dodge" \
    "7" \
    "3" \
    "1" \
    "F"

null device 
          1 


### Divide according to chromatin state

#### Obtain region enrichment details

In [11]:
%%bash
dirPATH=/mnt/Storage/home/yanghui/imprinting/result.2021/Exploring/CHMsInOtherContexts/CellStateTransition/CHMOrganization/Universal_specific/Features/Repeats
anPATH=/mnt/Storage/home/yanghui/annotations
shPATH=/mnt/Storage/home/yanghui/scripts/shell
mkdir -p ${dirPATH}/UniversalCHMs_NonCHMsCpGrich1kb;cd ${dirPATH}/UniversalCHMs_NonCHMsCpGrich1kb
 
for group in UniversalCHMs.cpm_A UniversalCHMs.cpm_B CpGrichNonCHMs.cpm_A CpGrichNonCHMs.cpm_B
do
    bash ${shPATH}/regionEnrichment_userDefined.sh \
        "${dirPATH}/UniversalCHMs_NonCHMsCpGrich1kb" \
        "mm10_euch" \
        "${anPATH}/mm10/mm10_euch.chrom.sizes" \
        "${group}.bed" \
        "${anPATH}/mm10/RegionEnrichment/mm10_euch.CGI_Unmasked.merged.bed;${anPATH}/mm10/RegionEnrichment/mm10_euch.promoter.merged.bed;${anPATH}/mm10/RegionEnrichment/mm10_euch.geneBody.merged.bed;${anPATH}/mm10/RegionEnrichment/mm10_euch.exon.merged.bed;${anPATH}/mm10/RegionEnrichment/mm10_euch.intron.merged.bed;${anPATH}/mm10/RegionEnrichment/mm10_euch.LTR.merged.bed;${anPATH}/mm10/RegionEnrichment/mm10_euch.LINE.merged.bed;${anPATH}/mm10/RegionEnrichment/mm10_euch.SINE.merged.bed" \
        "regionEnrich_${group}"
done # for group end

null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 


#### Plot together

In [13]:
%%bash
dirPATH=/mnt/Storage/home/yanghui/imprinting/result.2021/Exploring/CHMsInOtherContexts/CellStateTransition/CHMOrganization/Universal_specific/Features/Repeats
rPATH=/mnt/Storage/home/yanghui/scripts/R
cd ${dirPATH}/UniversalCHMs_NonCHMsCpGrich1kb

# Plot together
echo -e "xValue\tyValue\tgroup" > regionEnrich_cpm.ggplot_basic.txt
for group in UniversalCHMs.cpm_A UniversalCHMs.cpm_B CpGrichNonCHMs.cpm_A CpGrichNonCHMs.cpm_B
do
    tail -n +2 regionEnrich_${group}.EnrichScore.txt | \
        awk -v GROUP=${group} 'BEGIN{FS=OFS="\t"}{\
        split($1, a, "_");\
        print a[1], $2, GROUP;}' >> regionEnrich_cpm.ggplot_basic.txt
done # for group end

Rscript ${rPATH}/ggplot_barplot.r \
    "regionEnrich_cpm.ggplot_basic.txt" \
    "regionEnrich_cpm.ggplot_barplot.pdf" \
    "Region enrichment" \
    "" \
    "Enrichment score" \
    "-8;6.2;2" \
    "CGI;promoter;geneBody;exon;intron;LTR;LINE;SINE" \
    "UniversalCHMs.cpm_A;UniversalCHMs.cpm_B;CpGrichNonCHMs.cpm_A;CpGrichNonCHMs.cpm_B" \
    "dodge" \
    "7" \
    "3" \
    "1" \
    "F"

null device 
          1 


## Repeats subfamily enrichment

### Obtain enrichment score

In [92]:
%%bash
dirPATH=/mnt/Storage/home/yanghui/imprinting/result.2021/Exploring/CHMsInOtherContexts/CellStateTransition/CHMOrganization/Universal_specific/Features/Repeats
shPATH=/mnt/Storage/home/yanghui/scripts/shell
cd ${dirPATH}/UniversalCHMs_NonCHMsCpGrich1kb
mkdir -p LTR_Subfamily LINE_Subfamily SINE_Subfamily

for group in UniversalCHMs NonCHMsCpGrich
do
    for repeatType in LTR LINE SINE
    do
        bash ${shPATH}/regionEnrichment_${repeatType}_mm10_euch.sh \
            "${dirPATH}/UniversalCHMs_NonCHMsCpGrich1kb/${repeatType}_Subfamily" \
            "${dirPATH}/UniversalCHMs_NonCHMsCpGrich1kb/${group}.bed" \
            "${group}"
    done # for repeatType end
done # for group end

null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 

Runtime error (func=(main), adr=3): Divide by zero
Runtime error (func=(main), adr=3): Divide by zero


### Sina plot

#### Prepare for sina plot

In [7]:
%%bash
dirPATH=/mnt/Storage/home/yanghui/imprinting/result.2021/Exploring/CHMsInOtherContexts/CellStateTransition/CHMOrganization/Universal_specific/Features/Repeats

for repeatType in LTR LINE SINE
do
    cd ${dirPATH}/UniversalCHMs_NonCHMsCpGrich1kb/${repeatType}_Subfamily
    echo -e "xValue\tyValue\tgroup\tcol\tlabel" > enrichmentScore.ggplot_sina.txt
    for group in UniversalCHMs NonCHMsCpGrich
    do
        for fmlEnrich in $(ls ${group}.EnrichScore_${repeatType}_*.txt)
        do
            fmlName=$(echo ${fmlEnrich#*EnrichScore_${repeatType}_*})
            fmlName=$(echo ${fmlName%.txt})
            tail -n +2 ${fmlEnrich} | grep -v "inf" | \
                awk -v XVALUE=${group} -v GROUP=${repeatType} -v COL=${fmlName} 'BEGIN{FS=OFS="\t"}{\
                print XVALUE, $2, GROUP, COL, $1;}' >> enrichmentScore.ggplot_sina.txt
        done # for fmlEnrich end
    done # for group end
done # for repeatType end


#### Plot

In [8]:
%%bash
dirPATH=/mnt/Storage/home/yanghui/imprinting/result.2021/Exploring/CHMsInOtherContexts/CellStateTransition/CHMOrganization/Universal_specific/Features/Repeats
rPATH=/mnt/Storage/home/yanghui/scripts/R

LTR_sina(){
    cd ${dirPATH}/UniversalCHMs_NonCHMsCpGrich1kb/LTR_Subfamily
    Rscript ${rPATH}/ggplot_sina_colorfull.r \
        enrichmentScore.ggplot_sina.txt \
        enrichmentScore_filtered.ggplot_sina.pdf \
        "Enrichment score" \
        "-12;8;4" \
        "UniversalCHMs;NonCHMsCpGrich" \
        "LTR" \
        "ERV1;ERVK;ERVL;ERVL-MaLR;Gypsy" \
        "#CE0013;#C7A609;#64C0AB;#A14C94;#1187CD" \
        "0.2" \
        "0.75" \
        "1" \
        "5" \
        "5"
}
LTR_sina

LINE_sina(){
    cd ${dirPATH}/UniversalCHMs_NonCHMsCpGrich1kb/LINE_Subfamily
    Rscript ${rPATH}/ggplot_sina_colorfull.r \
        enrichmentScore.ggplot_sina.txt \
        enrichmentScore_filtered.ggplot_sina.pdf \
        "Enrichment score" \
        "-12;8;4" \
        "UniversalCHMs;NonCHMsCpGrich" \
        "LINE" \
        "CR1;Dong-R4;Jockey;L1;L1-Tx1;L2;Penelope;RTE-BovB;RTE-X" \
        "#CE0013;#16557A;#C7A609;#87C232;#64C0AB;#A14C94;#8B7E75;#1187CD;#000000" \
        "0.2" \
        "0.75" \
        "1" \
        "5" \
        "5"
}
LINE_sina

SINE_sina(){
    cd ${dirPATH}/UniversalCHMs_NonCHMsCpGrich1kb/SINE_Subfamily
    Rscript ${rPATH}/ggplot_sina_colorfull.r \
        enrichmentScore.ggplot_sina.txt \
        enrichmentScore_filtered.ggplot_sina.pdf \
        "Enrichment score" \
        "-12;8;4" \
        "UniversalCHMs;NonCHMsCpGrich" \
        "SINE" \
        "5S-Deu-L2;Alu;B2;B4;ID;MIR;tRNA;tRNA-Deu;tRNA-RTE" \
        "#CE0013;#16557A;#C7A609;#87C232;#64C0AB;#A14C94;#8B7E75;#1187CD;#000000" \
        "0.2" \
        "0.75" \
        "1" \
        "5" \
        "5"
}
SINE_sina

[1] "#CE0013" "#C7A609" "#64C0AB" "#A14C94" "#1187CD"
[1] "ERV1"      "ERVK"      "ERVL"      "ERVL-MaLR" "Gypsy"    

  1   2   3   4   5 
 89 255  67 101   8 
null device 
          1 
[1] "#CE0013" "#16557A" "#C7A609" "#87C232" "#64C0AB" "#A14C94" "#8B7E75"
[8] "#1187CD" "#000000"
[1] "CR1"      "Dong-R4"  "Jockey"   "L1"       "L1-Tx1"   "L2"       "Penelope"
[8] "RTE-BovB" "RTE-X"   

  1   4   6   7   8   9 
 12 180  15   2   2   3 
null device 
          1 
[1] "#CE0013" "#16557A" "#C7A609" "#87C232" "#64C0AB" "#A14C94" "#8B7E75"
[8] "#1187CD" "#000000"
[1] "5S-Deu-L2" "Alu"       "B2"        "B4"        "ID"        "MIR"      
[7] "tRNA"      "tRNA-Deu"  "tRNA-RTE" 

 1  2  3  4  5  6  7  8  9 
 1 31 10  8  6 10  1  1  2 
null device 
          1 


### Pie plot showing enriched subfamily

#### Prepare for pie plot

In [14]:
%%bash
dirPATH=/mnt/Storage/home/yanghui/imprinting/result.2021/Exploring/CHMsInOtherContexts/CellStateTransition/CHMOrganization/Universal_specific/Features/Repeats

for repeatType in LTR LINE SINE
do
    cd ${dirPATH}/UniversalCHMs_NonCHMsCpGrich1kb/${repeatType}_Subfamily
    for group in UniversalCHMs NonCHMsCpGrich
    do
        echo -e "value\tgroup" > ${group}.enriched.ggplot_pieplot.txt
        for fmlEnrich in $(ls ${group}.EnrichScore_${repeatType}_*.txt)
        do
            fmlName=$(echo ${fmlEnrich#*EnrichScore_${repeatType}_*})
            fmlName=$(echo ${fmlName%.txt})
            tail -n +2 ${fmlEnrich} | \
                awk -v GROUP=${fmlName} 'BEGIN{FS=OFS="\t";TOTAL=0;}{\
                if($2>=1){TOTAL+=1;}}END{print TOTAL, GROUP;}' >> ${group}.enriched.ggplot_pieplot.txt
        done # for fmlEnrich end
    done # for group end
done # for repeatType end

#### Plot

In [15]:
%%bash
dirPATH=/mnt/Storage/home/yanghui/imprinting/result.2021/Exploring/CHMsInOtherContexts/CellStateTransition/CHMOrganization/Universal_specific/Features/Repeats
rPATH=/mnt/Storage/home/yanghui/scripts/R

LTR_pie(){
    cd ${dirPATH}/UniversalCHMs_NonCHMsCpGrich1kb/LTR_Subfamily
    for group in UniversalCHMs NonCHMsCpGrich
    do
        Rscript ${rPATH}/ggplot_pie.r \
        ${group}.enriched.ggplot_pieplot.txt \
        ${group} \
        "ERV1;ERVK;ERVL;ERVL-MaLR;Gypsy" \
        "#CE0013;#C7A609;#64C0AB;#A14C94;#1187CD" \
        T \
        3.5 \
        3.5
    done # for group end
}
LTR_pie

LINE_pie(){
    cd ${dirPATH}/UniversalCHMs_NonCHMsCpGrich1kb/LINE_Subfamily
    for group in UniversalCHMs NonCHMsCpGrich
    do
        Rscript ${rPATH}/ggplot_pie.r \
        ${group}.enriched.ggplot_pieplot.txt \
        ${group} \
        "CR1;Dong-R4;Jockey;L1;L1-Tx1;L2;Penelope;RTE-BovB;RTE-X" \
        "#CE0013;#16557A;#C7A609;#87C232;#64C0AB;#A14C94;#8B7E75;#1187CD;#000000" \
        T \
        3.5 \
        3.5
    done # for group end
}
LINE_pie

SINE_pie(){
    cd ${dirPATH}/UniversalCHMs_NonCHMsCpGrich1kb/SINE_Subfamily
    for group in UniversalCHMs NonCHMsCpGrich
    do
        Rscript ${rPATH}/ggplot_pie.r \
        ${group}.enriched.ggplot_pieplot.txt \
        ${group} \
        "5S-Deu-L2;Alu;B2;B4;ID;MIR;tRNA;tRNA-Deu;tRNA-RTE" \
        "#CE0013;#16557A;#C7A609;#87C232;#64C0AB;#A14C94;#8B7E75;#1187CD;#000000" \
        T \
        3.5 \
        3.5
    done # for group end
}
SINE_pie

null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 


Warning messages:
1: Removed 9 rows containing missing values (position_stack). 
2: Removed 9 rows containing missing values (geom_text). 


### Scatter plot

In [16]:
%%bash
dirPATH=/mnt/Storage/home/yanghui/imprinting/result.2021/Exploring/CHMsInOtherContexts/CellStateTransition/CHMOrganization/Universal_specific/Features/Repeats
rPATH=/mnt/Storage/home/yanghui/scripts/R

LTR_F(){
    cd ${dirPATH}/UniversalCHMs_NonCHMsCpGrich1kb/LTR_Subfamily
    echo -e "xValue\tyValue\tgroup\tshape\tsubfamily" > enrichDetails.ggplot_scatter.txt # group: ERV1, ERVK...; shape: UniversalCHMs
    for group in UniversalCHMs
    do
        for fml in ERV1 ERVK
        do
            paste ${group}.overlaped_LTR_${fml}.txt ${group}.EnrichScore_LTR_${fml}.txt | \
                tail -n +2 | awk -v GROUP=${fml} -v SHAPE=${group} 'BEGIN{FS=OFS="\t"}{if($1==$4 && $5!="-inf"){print $3*100, $5, GROUP, SHAPE, $1;}}' >> enrichDetails.ggplot_scatter.txt
        done # for fml end
    done # for group end
     
    
    Rscript ${rPATH}/ggplot_scatterplot_basic.r \
        enrichDetails.ggplot_scatter.txt \
        enrichDetails \
        "Overlapped ratio (%)" \
        "Enrichment score" \
        "0;80;20" \
        "-12;8;4" \
        "ERV1;ERVK;ERVL;ERVL-MaLR;Gypsy" \
        "T;1;0" \
        "0.75" \
        "5" \
        "5" \
        "Set1"
}
LTR_F

null device 
          1 


### Conservation score

#### Obtain Enriched vs NonEnriched LTR subfamilies

In [17]:
%%bash
dirPATH=/mnt/Storage/home/yanghui/imprinting/result.2021/Exploring/CHMsInOtherContexts/CellStateTransition/CHMOrganization/Universal_specific/Features/Repeats

cd ${dirPATH}/UniversalCHMs_NonCHMsCpGrich1kb/LTR_Subfamily
cat UniversalCHMs.EnrichScore_LTR_*.txt | grep -v "EnrichScore" | \
    awk 'BEGIN{FS=OFS="\t";}{if($2!="-inf" && $2>=1){print $1;}}' | sort -u > UniversalCHMs.subfml_enriched.txt # 50
    
cat UniversalCHMs.EnrichScore_LTR_*.txt | grep -v "EnrichScore" | \
    awk 'BEGIN{FS=OFS="\t";}{if($2=="-inf" || $2<1){print $1;}}' | sort -u | grep -wvf UniversalCHMs.subfml_enriched.txt - > UniversalCHMs.subfml_nonenriched.txt # 592

#### Obtain corresponding annotation

In [19]:
%%bash
anPATH=/mnt/Storage/home/yanghui/annotations
dirPATH=/mnt/Storage/home/yanghui/imprinting/result.2021/Exploring/CHMsInOtherContexts/CellStateTransition/CHMOrganization/Universal_specific/Features/Repeats

cd ${dirPATH}/UniversalCHMs_NonCHMsCpGrich1kb/LTR_Subfamily

for group in enriched nonenriched
do
    for subfml in $(cut -f 1 UniversalCHMs.subfml_${group}.txt)
    do
        cat ${anPATH}/mm10/Repeats/LTR/*/mm10.${subfml}.bed | \
            intersectBed -a - -b ${anPATH}/mm10/mm10_euch.chrom.limits | \
            cut -f 1-3 >> UniversalCHMs.subfml_${group}.bed # 41,236 (enriched) || 837,636 (nonenriched)
    done # for subfml end
done # for group end

#### Average conservation score

In [20]:
%%bash
dirPATH=/mnt/Storage/home/yanghui/imprinting/result.2021/Exploring/CHMsInOtherContexts/CellStateTransition/CHMOrganization/Universal_specific/Features/Repeats
anPATH=/mnt/Storage/home/yanghui/annotations
shPATH=/mnt/Storage/home/yanghui/scripts/shell

cd ${dirPATH}/UniversalCHMs_NonCHMsCpGrich1kb/LTR_Subfamily
echo -e "xValue\tyValue\tgroup" > phastCons60wayPlacental.ggplot_basic.txt

for group in enriched nonenriched
do
    if [ -f UniversalCHMs.subfml_${group}.bed ]
    then
        bash ${shPATH}/averageSignalInRegion.sh \
            UniversalCHMs.subfml_${group}.bed \
            ${anPATH}/mm10/Conservation/mm10.60way.phastCons60wayPlacental.bw \
            1 \
            UniversalCHMs.subfml_${group}.phastConsPlacental.txt
        
        awk -v XVALUE=${group} 'BEGIN{FS=OFS="\t"}{print XVALUE, $4, "LTR";}' UniversalCHMs.subfml_${group}.phastConsPlacental.txt >> phastCons60wayPlacental.ggplot_basic.txt
    fi
done # for group end

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



#### Box plot

In [24]:
%%bash
dirPATH=/mnt/Storage/home/yanghui/imprinting/result.2021/Exploring/CHMsInOtherContexts/CellStateTransition/CHMOrganization/Universal_specific/Features/Repeats
anPATH=/mnt/Storage/home/yanghui/annotations
rPATH=/mnt/Storage/home/yanghui/scripts/R

cd ${dirPATH}/UniversalCHMs_NonCHMsCpGrich1kb/LTR_Subfamily
Rscript ${rPATH}/ggplot_boxplot.r \
    phastCons60wayPlacental.ggplot_basic.txt \
    phastCons60wayPlacental.ggplot_boxplot.pdf \
    LTR \
    "" \
    "Conservation score" \
    0 \
    0.4 \
    0.2 \
    "enriched;nonenriched" \
    "LTR" \
    3.5 \
    4.5


null device 
          1 
[1] "wilcox.test of significance between different xValues"
# A tibble: 1 × 8
  .y.    group1   group2          p p.adj p.format p.signif method  
  <chr>  <chr>    <chr>       <dbl> <dbl> <chr>    <chr>    <chr>   
1 yValue enriched nonenriched     0     0 <2e-16   ****     Wilcoxon
[1] "t.test of significance between different xValues"
# A tibble: 1 × 8
  .y.    group1   group2              p    p.adj p.format p.signif method
  <chr>  <chr>    <chr>           <dbl>    <dbl> <chr>    <chr>    <chr> 
1 yValue enriched nonenriched 5.42e-158 5.4e-158 <2e-16   ****     T-test


Warning messages:
1: Removed 24631 rows containing non-finite values (stat_boxplot). 
2: Removed 24631 rows containing non-finite values (stat_boxplot). 
3: Removed 1 rows containing missing values (geom_hline). 


#### Violin plot

In [26]:
%%bash
dirPATH=/mnt/Storage/home/yanghui/imprinting/result.2021/Exploring/CHMsInOtherContexts/CellStateTransition/CHMOrganization/Universal_specific/Features/Repeats
anPATH=/mnt/Storage/home/yanghui/annotations
rPATH=/mnt/Storage/home/yanghui/scripts/R

cd ${dirPATH}/UniversalCHMs_NonCHMsCpGrich1kb/LTR_Subfamily
Rscript ${rPATH}/ggplot_violin.r \
    phastCons60wayPlacental.ggplot_basic.txt \
    phastCons60wayPlacental.ggplot_violin.pdf \
    LTR \
    "" \
    "Conservation score" \
    0 \
    1 \
    0.2 \
    "enriched;nonenriched" \
    "LTR" \
    3.5 \
    4.5

null device 
          1 


## Overlap between universalCHMs and IAPEz-int subfamilies

### Two Universal CHMs groups (existence of IAPEz-int)

#### Groups

In [1]:
%%bash
dirPATH=/mnt/Storage/home/yanghui/imprinting/result.2021/Exploring/CHMsInOtherContexts/CellStateTransition/CHMOrganization/Universal_specific/Features/Repeats
anPATH=/mnt/Storage/home/yanghui/annotations

mkdir -p ${dirPATH}/UniversalCHMs_IAPEz-int;cd ${dirPATH}/UniversalCHMs_IAPEz-int
windowBed -w 2000 -u -a ${dirPATH}/UniversalCHMs.bed -b ${anPATH}/mm10/Repeats/LTR/ERVK/mm10_euch.IAPEz-int.merged.bed > overlapped.bed # 1,972
windowBed -w 2000 -v -a ${dirPATH}/UniversalCHMs.bed -b ${anPATH}/mm10/Repeats/LTR/ERVK/mm10_euch.IAPEz-int.merged.bed > notoverlapped.bed # 705

#### Length

In [4]:
%%bash
dirPATH=/mnt/Storage/home/yanghui/imprinting/result.2021/Exploring/CHMsInOtherContexts/CellStateTransition/CHMOrganization/Universal_specific/Features/Repeats
rPATH=/mnt/Storage/home/yanghui/scripts/R

mkdir -p ${dirPATH}/UniversalCHMs_IAPEz-int;cd ${dirPATH}/UniversalCHMs_IAPEz-int
# echo -e "xValue\tyValue\tgroup" > length.ggplot_basic.txt
# for group in overlapped notoverlapped
# do
#     awk -v XVALUE=${group} 'BEGIN{FS=OFS="\t"}{print XVALUE, $3-$2, "Length";}' ${group}.bed >> length.ggplot_basic.txt
# done # for group end

boxplot(){
    Rscript ${rPATH}/ggplot_boxplot.r \
        length.ggplot_basic.txt \
        length.ggplot_boxplot.pdf \
        Length \
        "" \
        "Length of universal CHMs" \
        0 \
        12000 \
        2000 \
        "overlapped;notoverlapped" \
        Length \
        3.5 \
        3.5
}
boxplot

violinplot(){
    Rscript ${rPATH}/ggplot_violin.r \
        length.ggplot_basic.txt \
        length.ggplot_violinplot.pdf \
        Length \
        "" \
        "Length of stable CHMs" \
        0 \
        12000 \
        2000 \
        "overlapped;notoverlapped" \
        Length \
        6 \
        3.5
}
violinplot

null device 
          1 
[1] "wilcox.test of significance between different xValues"
# A tibble: 1 × 8
  .y.    group1     group2               p   p.adj p.format p.signif method  
  <chr>  <chr>      <chr>            <dbl>   <dbl> <chr>    <chr>    <chr>   
1 yValue overlapped notoverlapped 3.70e-29 3.7e-29 <2e-16   ****     Wilcoxon
[1] "t.test of significance between different xValues"
# A tibble: 1 × 8
  .y.    group1     group2               p   p.adj p.format p.signif method
  <chr>  <chr>      <chr>            <dbl>   <dbl> <chr>    <chr>    <chr> 
1 yValue overlapped notoverlapped 2.59e-19 2.6e-19 <2e-16   ****     T-test
null device 
          1 


Warning messages:
1: Removed 10 rows containing non-finite values (stat_boxplot). 
2: Removed 10 rows containing non-finite values (stat_boxplot). 
3: Removed 1 rows containing missing values (geom_hline). 
Warning messages:
1: Removed 10 rows containing non-finite values (stat_ydensity). 
2: Removed 10 rows containing non-finite values (stat_boxplot). 


#### Region enrichment

In [6]:
%%bash
dirPATH=/mnt/Storage/home/yanghui/imprinting/result.2021/Exploring/CHMsInOtherContexts/CellStateTransition/CHMOrganization/Universal_specific/Features/Repeats
anPATH=/mnt/Storage/home/yanghui/annotations
shPATH=/mnt/Storage/home/yanghui/scripts/shell
mkdir -p ${dirPATH}/UniversalCHMs_IAPEz-int;cd ${dirPATH}/UniversalCHMs_IAPEz-int

for group in overlapped notoverlapped
do
    bash ${shPATH}/regionEnrichment_userDefined.sh \
        "${dirPATH}/UniversalCHMs_IAPEz-int" \
        "mm10_euch" \
        "${anPATH}/mm10/mm10_euch.chrom.sizes" \
        "${group}.bed" \
        "${anPATH}/mm10/RegionEnrichment/mm10_euch.CGI_Unmasked.merged.bed;${anPATH}/mm10/RegionEnrichment/mm10_euch.promoter.merged.bed;${anPATH}/mm10/RegionEnrichment/mm10_euch.geneBody.merged.bed;${anPATH}/mm10/RegionEnrichment/mm10_euch.exon.merged.bed;${anPATH}/mm10/RegionEnrichment/mm10_euch.intron.merged.bed;${anPATH}/mm10/RegionEnrichment/mm10_euch.LTR.merged.bed;${anPATH}/mm10/RegionEnrichment/mm10_euch.LINE.merged.bed;${anPATH}/mm10/RegionEnrichment/mm10_euch.SINE.merged.bed" \
        "regionEnrich_${group}"
done # for group end

null device 
          1 
null device 
          1 
null device 
          1 
null device 
          1 
